# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


### Libraries

In [ ]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 5.6 MB/s eta 0:00:00


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/CV/Project/Code')
from models import GbcNet

import random
import json
import torch
import pickle
import os
import torch.nn as nn
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2

import copy
import time
import math
import cv2
from PIL import Image
import torch.nn.functional as F
from torchvision.transforms import Resize, ToTensor, Compose, GaussianBlur
from torchvision.models.detection import retinanet_resnet50_fpn_v2
from torchvision.models.detection import RetinaNet
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
import torch.nn.init as init
from torch.optim import SGD, Adam, NAdam
from torchmetrics.detection import MeanAveragePrecision
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.functional import one_hot

from sklearn.metrics import roc_auc_score

from torchvision.utils import draw_bounding_boxes
import torchvision
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### File Paths

In [ ]:
!unzip "/content/drive/MyDrive/CV/Project/data/images.zip" -d "/content"
imgs_folder_path = "/content/images"
train_txt_path = "/content/drive/MyDrive/CV/Project/data/train.txt" # Original train dataset
#train_txt_path = "/content/drive/MyDrive/CV/Project/data/augmented_train.txt" # Flipped malignant images for total balanced dataset
#train_txt_path = "/content/drive/MyDrive/CV/Project/data/train_double_malignant.txt" # Rotated malignant images for total balanced dataset
#train_txt_path = "/content/drive/MyDrive/CV/Project/data/train_nr.txt" # Only rotated normal images to be balanced with sum of malignant and benign
#train_txt_path = "/content/drive/MyDrive/CV/Project/data/train_nmr.txt" # rotated malignanet to be equal to benign, rotated normal to be equal to benign+malignant, 3 and 6 degrees rotation
#train_txt_path = "/content/drive/MyDrive/CV/Project/data/train_nmr_2.txt" # rotated malignanet to be equal to benign, rotated normal to be equal to benign+malignant, 3 and -3 degrees rotation
val_txt_path = "/content/drive/MyDrive/CV/Project/data/test.txt"
gold_file_path = "/content/drive/MyDrive/CV/Project/data/gold_boxes.json"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"

#train_txt_path = "/content/drive/MyDrive/CV/Project/data/train val test split/temp.txt"
#val_txt_path = "/content/drive/MyDrive/CV/Project/data/train val test split/temp.txt"

### Helper Functions

In [ ]:
def read_txt(txt_path):
  ids = []
  cls = []
  with open(txt_path,"r") as f:
    for line in f.readlines():
      ids.append(line.strip().split(',')[0])
      cls.append(int(line.strip().split(',')[1]))

  return ids, cls

def read_json(json_path):
  with open(json_path, "r") as f:
    json_file = json.load(f)

  return json_file

import matplotlib.pyplot as plt
import matplotlib.patches as patches

def visualize(img, pred_bboxes=None, title=None):
    print("visualize")
    edgecolors = ["y", "r", "c"]
    fig, ax = plt.subplots(figsize=(10, 5))

    # Plot the original image with bounding box
    if torch.is_tensor(img):
      ax.imshow(img.permute(1, 2, 0).numpy())
    else:
      ax.imshow(img)
    if pred_bboxes is not None:
      for i in range(len(pred_bboxes)):
        pred_bbox = pred_bboxes[i]
        rect = patches.Rectangle((pred_bbox[0], pred_bbox[1]), pred_bbox[2]-pred_bbox[0], pred_bbox[3]-pred_bbox[1], linewidth=2, edgecolor=edgecolors[i], facecolor='none')
        ax.add_patch(rect)
    ax.set_title(title)

    plt.show()


def gold_box_proportion(x):
  if x < 30:
    return 1
  elif 30 <= x < 35:
    return 0.75
  elif 35 <= x < 40:
    return 0.5
  elif 40 <= x < 45:
    return 0.25
  else:
    return 0

def print_status(txt, status_flag):
  if status_flag==False:
    print(txt)
  return True

def filter_background(predictions):
  # Filter out the predictions with label 0
  filtered_predictions = []
  for prediction in predictions:
      mask = prediction['labels'] != 0
      filtered_prediction = {k: v[mask] for k, v in prediction.items()}
      filtered_predictions.append(filtered_prediction)
  return filtered_predictions

def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

### Transforms

In [ ]:
class img_bbox_transform(object):
    def __init__(self, target_width, target_height):
      self.target_width = target_width
      self.target_height = target_height
      self.transforms = Compose([Resize((self.target_width, self.target_height)),
                                ToTensor()])

    def __call__(self, image=None, bbox = None, current_width=None, current_height=None):
      transformed_image = None
      if image is not None:
        current_width, current_height = image.size
        transformed_image = self.transforms(image)
      transformed_bbox = None
      if bbox is not None:
        transformed_bbox = self.transform_bbox(bbox, current_width, current_height)
      return transformed_image, transformed_bbox

    def transform_bbox(self, bbox, current_width=None, current_height=None):
      new_x1 = bbox[0]*self.target_width/current_width
      new_y1 = bbox[1]*self.target_height/current_height
      new_x2 = bbox[2]*self.target_width/current_width
      new_y2 = bbox[3]*self.target_height/current_height
      return [round(new_x1.item()), round(new_y1.item()), round(new_x2.item()), round(new_y2.item())]


def crop_image(image, box, p):
    x1, y1, x2, y2 = box
    # Input images in CHW
    cropped_image = image[:, int((1-p)*y1):int((1+p)*y2), int((1-p)*x1):int((1+p)*x2)]
    return cropped_image

def get_train_transforms(width, height, p=None):
    return A.Compose([
        A.Rotate(limit=5, p=0.5, border_mode=cv2.BORDER_CONSTANT, value=0),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=0, p=0.6, border_mode=cv2.BORDER_CONSTANT, value=0),
        A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit= 0.1, p=0.5),
        A.HueSaturationValue(hue_shift_limit=3, sat_shift_limit=5, val_shift_limit=0, p=0.5),
        A.MotionBlur(blur_limit=(3, 5), p=0.5),
        A.Perspective(scale=(0.01, 0.05), keep_size=True, pad_mode=0, pad_val=0, mask_pad_val=0, fit_output=False, interpolation=1, always_apply=False, p=0.7),
        A.MultiplicativeNoise(multiplier=(0.9, 1.1), p=0.5),
        A.Resize(height=height, width=width, p=1),
        ToTensorV2(p=1.0)
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

def get_eval_transforms(width, height):
    return A.Compose([
        A.Resize(height=height, width=width, p=1),
        ToTensorV2(p=1.0)
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

### Dataset

In [ ]:
class InputDataset(Dataset):
    def __init__(self, img_folder, id_txt, gold_json, p=0.15, width=224, height=224, img_transforms=None, do_crop=None, predicted_bboxes_json=None, to_blur=False, blur_kernel_size=(65,65), sigma=0):
        self.img_folder = img_folder
        self.id_txt = id_txt
        self.gold_json = gold_json
        self.to_blur = to_blur
        self.blur_kernel_size = blur_kernel_size
        self.sigma = sigma
        self.do_crop=do_crop
        self.p=p
        self.width=width
        self.height=height
        self.transforms = img_transforms
        self.ids, self.true_labels = read_txt(id_txt)
        self.gold_dict = read_json(gold_json)
        self.predicted_boxes = None
        if predicted_bboxes_json is not None:
          self.predicted_boxes=read_json(predicted_bboxes_json)

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        img_id = self.ids[idx]
        img_label = self.true_labels[idx]
        gold_bbox = self.gold_dict[img_id]['Gold']
        img_path = f"{self.img_folder}/{img_id}"
        img = cv2.imread(img_path)
        img_height, img_width, _ = img.shape
        if self.to_blur:
          img = cv2.GaussianBlur(img, self.blur_kernel_size, self.sigma)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if (self.do_crop==None) or (self.do_crop=="gold_boxes"):
          transformed_dict = self.transforms(image=img, bboxes=[gold_bbox], labels=[1])
        elif self.do_crop=="pred_boxes":
          pred_bbox = self.predicted_boxes[img_id]['Boxes']
          transformed_dict = self.transforms(image=img, bboxes=[pred_bbox], labels=[1])

        #visualize(img, pred_bboxes=[gold_bbox, pred_bbox], title=f"raw image {img_id}")
        transformed_img = transformed_dict['image'].float().div(255)
        transformed_bboxes = [round(x) for x in transformed_dict['bboxes'][0]]
        #visualize(transformed_img, [[0,0,0,0], transformed_bboxes], title=f"transformed img {img_id}")

        if self.do_crop is not None:
          transformed_bboxes_area = (transformed_bboxes[2]-transformed_bboxes[0]) * (transformed_bboxes[3]-transformed_bboxes[1])
          if transformed_bboxes_area > 0:
            transformed_img = crop_image(transformed_img, transformed_bboxes, p=self.p)
            transformed_img = F.interpolate(transformed_img.unsqueeze(0), size=(self.height, self.width), mode='bilinear', align_corners=False).squeeze(0)

        #visualize(transformed_img, title=f"cropped img {img_id}")

        target = {}
        target['boxes'] = torch.tensor(transformed_bboxes).unsqueeze(0)
        target['labels'] = torch.tensor(1).unsqueeze(0)
        target['true_labels'] = torch.tensor(img_label, dtype=torch.int64)
        target['image shape'] = {'height': img_height, 'width': img_width}
        target['Gold'] = gold_bbox
        return img_id, transformed_img, target

def collate_fn(batch):
  images_ids, images, targets = zip(*batch)
  return list(images_ids), list(images), list(targets)

### Evaluation

In [ ]:
def compute_IoU(boxA, boxB):
  x1_A, y1_A, x2_A, y2_A = boxA
  x1_B, y1_B, x2_B, y2_B = boxB

  x1 = max(x1_A, x1_B)
  y1 = max(y1_A, y1_B)
  x2 = min(x2_A, x2_B)
  y2 = min(y2_A, y2_B)

  if x2 <= x1 or y2 <= y1:
    return torch.tensor(0).to(device)

  intersection_area = (x2 - x1) * (y2 - y1)
  boxA_area = (x2_A - x1_A) * (y2_A - y1_A)
  boxB_area = (x2_B - x1_B) * (y2_B - y1_B)

  iou = intersection_area / float(boxA_area + boxB_area - intersection_area)
  return iou.float()

def center_within_gold(box, gold):
  x_center = (box[0] + box[2])/2
  y_center = (box[1] + box[3])/2
  x_min_gold, x_max_gold = gold[0], gold[2]
  y_min_gold, y_max_gold = gold[1], gold[3]
  if (x_min_gold<x_center<x_max_gold) and (y_min_gold<y_center<y_max_gold):
    return True
  else:
    return False

def compute_bbox_metrics(targets, predictions):
  tp = 0
  fp = 0
  fn = 0
  for i in range(len(predictions)):
    target_box = targets[i]['boxes'][0]
    pred_boxes = predictions[i]['boxes']
    if len(pred_boxes) > 0:
      for pred_box in pred_boxes:
        if center_within_gold(pred_box, target_box):
          tp += 1
        else:
          fp += 1
    else:
      fn += 1
  return tp, fp, fn

def evaluate_IoUs(targets, predictions):
  IoUs = []
  for i in range(len(predictions)):
    target_box = targets[i]['boxes'][0]
    pred_boxes = predictions[i]['boxes']
    if len(pred_boxes) > 0:
      for pred_box in pred_boxes:
        IoUs.append(compute_IoU(target_box, pred_box.squeeze(0)))
    else:
      IoUs.append(torch.tensor(0.0).to(device))
  return IoUs

# Retinanet

## Evaluation

In [ ]:
def evaluate_model(model, dataset_loader, width=224, height=224, return_bboxes=False):
    model.eval()
    all_IoUs = []
    TPs = 0
    FNs = 0
    FPs = 0
    precision = 0
    recall = 0
    all_bboxes= {}
    mAP_metric = MeanAveragePrecision(box_format='xyxy', iou_type='bbox')
    with torch.inference_mode():
      for i, data in enumerate(dataset_loader, 0):
        images_ids, inputs, targets = data[0],\
                                      [image.to(device) for image in data[1]],\
                                      [{k: v.to(device) if torch.is_tensor(v) else v for k, v in t.items()} for t in data[2]]

        predictions = model(inputs)
        foreground_preds = filter_background(predictions)
        IoUs = evaluate_IoUs(targets, foreground_preds)
        all_IoUs += IoUs
        mAP_metric.update(foreground_preds, targets)

        tp, fp, fn = compute_bbox_metrics(targets, foreground_preds)
        TPs += tp
        FPs += fp
        FNs += fn

        if return_bboxes:
          for i in range(len(targets)):
            fname = images_ids[i]
            target_bbox = targets[i]['Gold']
            bbox_transform = img_bbox_transform(target_height=targets[i]['image shape']['height'], target_width=targets[i]['image shape']['width'])
            pred_bbox = foreground_preds[i]['boxes'].squeeze(0)
            _, pred_bbox = bbox_transform(bbox=pred_bbox, current_width=width, current_height=height)
            all_bboxes[fname] = {"Boxes": pred_bbox, "Gold": target_bbox}

    mIoU = torch.stack(all_IoUs).mean()
    medIoU = torch.stack(all_IoUs).median()
    mAP_dict = mAP_metric.compute()
    mAP = mAP_dict['map']
    print(f"mIoU = {mIoU}, median IoU: {medIoU}, mAP = {round(mAP_dict['map'].item(),3)}, mAP_50 = {round(mAP_dict['map_50'].item(),3)}, mAP_75 = {round(mAP_dict['map_75'].item(),3)}")

    if (TPs + FPs) != 0:
      precision = TPs / (TPs + FPs)
    if (TPs + FNs) != 0:
      recall = TPs / (TPs + FNs)
    print(f"TP: {TPs}, FN: {FNs}, FP: {FPs}")
    print("Precision: {}\nRecall: {}".format(precision, recall))
    return mAP, all_bboxes

## Training

In [ ]:
def train_model(model, save_model_dir, num_epochs, optimizer, best_mAP=0, last_epoch=0, train_transform=None, eval_transform=None,
                num_worse_epochs=0, batch_size=10, patience=None, checkpoint_save_rate=5, lr_scheduler = None, train_scores=[], val_scores=[]):

    # Load train set
    train_data = InputDataset(imgs_folder_path, train_txt_path, gold_file_path, img_transforms=train_transform)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

    # Load validation set
    val_data = InputDataset(imgs_folder_path, val_txt_path, gold_file_path, img_transforms=eval_transform)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    best_epoch = 0
    best_model = None

    os.makedirs(f'{save_model_dir}/checkpoints', exist_ok=True)
    if patience is not None:
      os.makedirs(f'{save_model_dir}/Final Models', exist_ok=True)

    print(f"Best mAP so far: {best_mAP}")

    for epoch in range(last_epoch+1, num_epochs+1):

      print(f'\nEpoch {epoch}:')
      start_time = time.time()

      ### Training Loop
      model.train()
      running_loss = 0.0
      total_step = len(train_loader)
      for i, data in enumerate(train_loader, 0):
          images_ids, inputs, targets = data[0],\
                                        [image.to(device) for image in data[1]],\
                                        [{k: v.to(device) if torch.is_tensor(v) else v for k, v in t.items()} for t in data[2]]

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward pass
          loss_dict = model(inputs, targets)

          # loss
          losses = sum(loss for loss in loss_dict.values())
          running_loss += losses

          # loss backward
          losses.backward()

          # optimize(update weights)
          optimizer.step()

      train_loss = running_loss/total_step
      print(f"Train Loss: {train_loss}, Total step: {total_step}")

      model.eval()
      ### Evaluation Loop (training data)
      print("Training evaluation:")
      train_mAP, _ = evaluate_model(model, train_loader, return_bboxes=False)
      train_scores.append(train_mAP)

      ### Evaluation Loop (validation data)
      print("Validation evaluation:")
      val_mAP, _ = evaluate_model(model, val_loader, return_bboxes=False)
      val_scores.append(val_mAP)

      # Early Stop
      if val_mAP > best_mAP:
        best_mAP = val_mAP
        best_epoch = epoch
        best_model = copy.deepcopy(model)
        num_worse_epochs = 0
      else:
        num_worse_epochs += 1
        if patience is not None:
          if num_worse_epochs >= patience:
            # save the best model and its optimizer
            if best_model is not None:
              torch.save({'model_state_dict': best_model.state_dict(),
                          'optimizer_state_dict': optimizer.state_dict(),
                          'best_mAP': best_mAP,
                          'epoch': best_epoch},
                          f'{save_model_dir}/Final Models/best_model_epoch_{best_epoch}.pth')
              print(f'Best model is saved to: {save_model_dir}/Final Models/best_model_epoch_{best_epoch}.pth')
              # save the last model and its optimizer
              torch.save({'model_state_dict': model.state_dict(),
                          'optimizer_state_dict': optimizer.state_dict(),
                          'best_mAP': best_mAP,
                          'epoch': epoch,
                          'num_worse_epochs': num_worse_epochs},
                          f'{save_model_dir}/Final Models/last_model_epoch_{epoch}.pth')
              print(f'Last model is saved to: {save_model_dir}/Final Models/last_model_epoch_{epoch}.pth')

            else:
              print("\nNo model to save!")

            print(f"No better results for last {patience} epochs!\nBest model was for epoch {best_epoch} with best mAP: {best_mAP}")
            break


      if (epoch) % checkpoint_save_rate == 0:
        random_state = random.getstate()
        np_random_state = np.random.get_state()
        torch_random_state = torch.get_rng_state()
        torch.save({'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'best_mAP': best_mAP,
                    'train_scores': train_scores,
                    'val_scores': val_scores,
                    'epoch': epoch,
                    'num_worse_epochs': num_worse_epochs,
                    'random_state': random_state,
                    'np_random_state': np_random_state,
                    'torch_random_state': torch_random_state},
                    f'{save_model_dir}/checkpoints/checkpoint_epoch_{epoch}.pth')
        print(f"\nCheckpoint is saved to: {save_model_dir}/checkpoints/checkpoint_epoch_{epoch}.pth\n")

        if best_model is not None:
          torch.save({'model_state_dict': best_model.state_dict(),
                      'optimizer_state_dict': optimizer.state_dict(),
                      'best_mAP': best_mAP,
                      'epoch': best_epoch},
                      f'{save_model_dir}/checkpoints/best_model_epoch_{best_epoch}.pth')
          print(f'Best model is saved to: {save_model_dir}/checkpoints/best_model_epoch_{best_epoch}.pth')
          print(f"Best model was for epoch {best_epoch} with best mAP: {best_mAP}")

        else:
          print("\nNo Best model to save!")


      # Time
      end_time = time.time()
      elapsed_time = end_time - start_time
      print(f"Epoch time: {elapsed_time}")
      remaining_epochs = num_epochs - epoch - 1
      remaining_time = round(elapsed_time * remaining_epochs)
      hours, remainder = divmod(remaining_time, 3600)
      minutes, seconds = divmod(remainder, 60)
      print(f"Estimated time remaining: {hours}:{minutes}:{seconds}")

      if lr_scheduler is not None:
        lr_scheduler.step()


## Experiments

### Retinanet with GBUSV weights

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 200
batch_size = 55
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/RetinaNet/RetinaNet with GBUSV weights/retinanet_gbusv_backbone_weights.pth"

retinanet = retinanet_resnet50_fpn_v2(num_classes=2, detections_per_img=1, score_thresh=0.05, nms_thresh=0.5, min_size=min_size, max_size=max_size).to(device)
retinanet.load_state_dict(torch.load(retinanet_weight_path, map_location = device))
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, GBUSV"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler)

Best mAP so far: 0

Epoch 1:
Train Loss: 1.3312163352966309, Total step: 21
Training evaluation:
mIoU = 0.3862794041633606, median IoU: 0.37769514322280884, mAP = 0.019, mAP_50 = 0.096, mAP_75 = 0.003
TP: 889, FN: 0, FP: 244
Precision: 0.7846425419240953
Recall: 1.0
Validation evaluation:
mIoU = 0.31986796855926514, median IoU: 0.3014960289001465, mAP = 0.01, mAP_50 = 0.05, mAP_75 = 0.0
TP: 78, FN: 0, FP: 44
Precision: 0.639344262295082
Recall: 1.0
Epoch time: 624.5996792316437
Estimated time remaining: 34:21:11

Epoch 2:
Train Loss: 0.8465482592582703, Total step: 21
Training evaluation:
mIoU = 0.4810961186885834, median IoU: 0.49202248454093933, mAP = 0.068, mAP_50 = 0.307, mAP_75 = 0.008
TP: 1009, FN: 0, FP: 124
Precision: 0.8905560458958517
Recall: 1.0
Validation evaluation:
mIoU = 0.3982087969779968, median IoU: 0.3731516897678375, mAP = 0.04, mAP_50 = 0.185, mAP_75 = 0.006
TP: 94, FN: 0, FP: 28
Precision: 0.7704918032786885
Recall: 1.0
Epoch time: 132.65509343147278
Estimated tim

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 200
batch_size = 55
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, GBUSV"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

retinanet = retinanet_resnet50_fpn_v2(num_classes=2, detections_per_img =1, score_thresh = 0.05, nms_thresh = 0.5, min_size=min_size, max_size=max_size).to(device)
params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, GBUSV/checkpoints/checkpoint_epoch_55.pth", map_location=device)
retinanet.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
best_mAP = checkpoint['best_mAP']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler, last_epoch=last_epoch, best_mAP=best_mAP, train_scores=train_scores, val_scores=val_scores)

Best mAP so far: 0.24966102838516235

Epoch 56:
Train Loss: 0.22709307074546814, Total step: 21
Training evaluation:
mIoU = 0.8376189470291138, median IoU: 0.8474921584129333, mAP = 0.672, mAP_50 = 0.99, mAP_75 = 0.847
TP: 1130, FN: 0, FP: 3
Precision: 0.9973521624007061
Recall: 1.0
Validation evaluation:
mIoU = 0.5929762721061707, median IoU: 0.6324926018714905, mAP = 0.231, mAP_50 = 0.659, mAP_75 = 0.096
TP: 111, FN: 0, FP: 11
Precision: 0.9098360655737705
Recall: 1.0
Epoch time: 926.7759048938751
Estimated time remaining: 36:48:49

Epoch 57:
Train Loss: 0.22770854830741882, Total step: 21
Training evaluation:
mIoU = 0.8246161937713623, median IoU: 0.8351309895515442, mAP = 0.642, mAP_50 = 0.99, mAP_75 = 0.806
TP: 1131, FN: 0, FP: 2
Precision: 0.9982347749338041
Recall: 1.0
Validation evaluation:
mIoU = 0.5928422212600708, median IoU: 0.6305555105209351, mAP = 0.231, mAP_50 = 0.664, mAP_75 = 0.102
TP: 113, FN: 0, FP: 9
Precision: 0.9262295081967213
Recall: 1.0
Epoch time: 139.7862441

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 200
batch_size = 55
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, GBUSV"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

retinanet = retinanet_resnet50_fpn_v2(num_classes=2, detections_per_img =1, score_thresh = 0.05, nms_thresh = 0.5, min_size=min_size, max_size=max_size).to(device)
params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, GBUSV/checkpoints/checkpoint_epoch_85.pth", map_location=device)
retinanet.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
best_mAP = checkpoint['best_mAP']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler, last_epoch=last_epoch, best_mAP=best_mAP, train_scores=train_scores, val_scores=val_scores)

Best mAP so far: 0.2631535232067108

Epoch 86:
Train Loss: 0.1782258301973343, Total step: 21
Training evaluation:
mIoU = 0.8569037914276123, median IoU: 0.8703222870826721, mAP = 0.714, mAP_50 = 0.99, mAP_75 = 0.914
TP: 1132, FN: 0, FP: 1
Precision: 0.999117387466902
Recall: 1.0
Validation evaluation:
mIoU = 0.6038494110107422, median IoU: 0.6431392431259155, mAP = 0.257, mAP_50 = 0.729, mAP_75 = 0.116
TP: 115, FN: 0, FP: 7
Precision: 0.9426229508196722
Recall: 1.0
Epoch time: 1023.1718587875366
Estimated time remaining: 32:6:58

Epoch 87:
Train Loss: 0.180371955037117, Total step: 21
Training evaluation:
mIoU = 0.85929274559021, median IoU: 0.8709596991539001, mAP = 0.716, mAP_50 = 0.99, mAP_75 = 0.916
TP: 1133, FN: 0, FP: 0
Precision: 1.0
Recall: 1.0
Validation evaluation:
mIoU = 0.6067188382148743, median IoU: 0.6450549364089966, mAP = 0.251, mAP_50 = 0.706, mAP_75 = 0.106
TP: 115, FN: 0, FP: 7
Precision: 0.9426229508196722
Recall: 1.0
Epoch time: 136.84552311897278
Estimated time 

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 200
batch_size = 55
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, GBUSV"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

retinanet = retinanet_resnet50_fpn_v2(num_classes=2, detections_per_img =1, score_thresh = 0.05, nms_thresh = 0.5, min_size=min_size, max_size=max_size).to(device)
params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, GBUSV/checkpoints/checkpoint_epoch_115.pth", map_location=device)
retinanet.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
best_mAP = checkpoint['best_mAP']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler, last_epoch=last_epoch, best_mAP=best_mAP, train_scores=train_scores, val_scores=val_scores)

Best mAP so far: 0.2760861814022064

Epoch 116:
Train Loss: 0.16295400261878967, Total step: 21
Training evaluation:
mIoU = 0.8792145252227783, median IoU: 0.8919958472251892, mAP = 0.762, mAP_50 = 0.99, mAP_75 = 0.951
TP: 1132, FN: 0, FP: 1
Precision: 0.999117387466902
Recall: 1.0
Validation evaluation:
mIoU = 0.6070996522903442, median IoU: 0.6359270811080933, mAP = 0.257, mAP_50 = 0.694, mAP_75 = 0.11
TP: 112, FN: 0, FP: 10
Precision: 0.9180327868852459
Recall: 1.0
Epoch time: 524.1873753070831
Estimated time remaining: 12:5:8

Epoch 117:
Train Loss: 0.15858028829097748, Total step: 21
Training evaluation:
mIoU = 0.8674870133399963, median IoU: 0.8781323432922363, mAP = 0.738, mAP_50 = 0.99, mAP_75 = 0.95
TP: 1132, FN: 0, FP: 1
Precision: 0.999117387466902
Recall: 1.0
Validation evaluation:
mIoU = 0.6084433794021606, median IoU: 0.6359184384346008, mAP = 0.262, mAP_50 = 0.723, mAP_75 = 0.142
TP: 113, FN: 0, FP: 9
Precision: 0.9262295081967213
Recall: 1.0
Epoch time: 133.632405042648

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 200
batch_size = 55
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, GBUSV"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

retinanet = retinanet_resnet50_fpn_v2(num_classes=2, detections_per_img =1, score_thresh = 0.05, nms_thresh = 0.5, min_size=min_size, max_size=max_size).to(device)
params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, GBUSV/checkpoints/checkpoint_epoch_145.pth", map_location=device)
retinanet.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
best_mAP = checkpoint['best_mAP']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler, last_epoch=last_epoch, best_mAP=best_mAP, train_scores=train_scores, val_scores=val_scores)

Best mAP so far: 0.2760861814022064

Epoch 146:
Train Loss: 0.14503711462020874, Total step: 21
Training evaluation:
mIoU = 0.8859321475028992, median IoU: 0.8968210220336914, mAP = 0.781, mAP_50 = 0.99, mAP_75 = 0.968
TP: 1132, FN: 0, FP: 1
Precision: 0.999117387466902
Recall: 1.0
Validation evaluation:
mIoU = 0.6081435084342957, median IoU: 0.6500564217567444, mAP = 0.265, mAP_50 = 0.747, mAP_75 = 0.118
TP: 113, FN: 0, FP: 9
Precision: 0.9262295081967213
Recall: 1.0
Epoch time: 1096.4527742862701
Estimated time remaining: 16:8:32

Epoch 147:
Train Loss: 0.14718948304653168, Total step: 21
Training evaluation:
mIoU = 0.8726078867912292, median IoU: 0.8818114995956421, mAP = 0.753, mAP_50 = 0.99, mAP_75 = 0.968
TP: 1132, FN: 0, FP: 1
Precision: 0.999117387466902
Recall: 1.0
Validation evaluation:
mIoU = 0.6026763319969177, median IoU: 0.6504306197166443, mAP = 0.254, mAP_50 = 0.752, mAP_75 = 0.103
TP: 114, FN: 0, FP: 8
Precision: 0.9344262295081968
Recall: 1.0
Epoch time: 138.349706411

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 200
batch_size = 55
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, GBUSV"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

retinanet = retinanet_resnet50_fpn_v2(num_classes=2, detections_per_img =1, score_thresh = 0.05, nms_thresh = 0.5, min_size=min_size, max_size=max_size).to(device)
params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, GBUSV/checkpoints/checkpoint_epoch_175.pth", map_location=device)
retinanet.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
best_mAP = checkpoint['best_mAP']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler, last_epoch=last_epoch, best_mAP=best_mAP, train_scores=train_scores, val_scores=val_scores)

Best mAP so far: 0.2760861814022064

Epoch 176:
Train Loss: 0.1362682729959488, Total step: 21
Training evaluation:
mIoU = 0.8939946293830872, median IoU: 0.9050693511962891, mAP = 0.796, mAP_50 = 0.99, mAP_75 = 0.969
TP: 1133, FN: 0, FP: 0
Precision: 1.0
Recall: 1.0
Validation evaluation:
mIoU = 0.6101413369178772, median IoU: 0.6373836398124695, mAP = 0.271, mAP_50 = 0.747, mAP_75 = 0.116
TP: 113, FN: 0, FP: 9
Precision: 0.9262295081967213
Recall: 1.0
Epoch time: 546.7195281982422
Estimated time remaining: 3:29:35

Epoch 177:
Train Loss: 0.13238050043582916, Total step: 21
Training evaluation:
mIoU = 0.8820937275886536, median IoU: 0.8910084366798401, mAP = 0.772, mAP_50 = 0.99, mAP_75 = 0.979
TP: 1133, FN: 0, FP: 0
Precision: 1.0
Recall: 1.0
Validation evaluation:
mIoU = 0.6104258298873901, median IoU: 0.6484502553939819, mAP = 0.271, mAP_50 = 0.732, mAP_75 = 0.125
TP: 112, FN: 0, FP: 10
Precision: 0.9180327868852459
Recall: 1.0
Epoch time: 138.43775844573975
Estimated time remainin

### Retinanet with GBUSV weights, with 3 trainable layers

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 200
batch_size = 55
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/RetinaNet/RetinaNet with GBUSV weights/retinanet_gbusv_backbone_weights.pth"

retinanet = retinanet_resnet50_fpn_v2(num_classes=2, detections_per_img=1, score_thresh=0.05, nms_thresh=0.5, min_size=min_size, max_size=max_size).to(device)
retinanet.load_state_dict(torch.load(retinanet_weight_path, map_location = device))

# Freeze all layers in the backbone
for param in retinanet.backbone.body.parameters():
    param.requires_grad = False

# Unfreeze the last two layers in the backbone
for param in retinanet.backbone.body.layer4.parameters():
    param.requires_grad = True
for param in retinanet.backbone.body.layer3.parameters():
    param.requires_grad = True
for param in retinanet.backbone.body.layer2.parameters():
    param.requires_grad = True

save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, GBUSV, 3 trainable layers"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler)

Best mAP so far: 0

Epoch 1:
Train Loss: 1.3317416906356812, Total step: 21
Training evaluation:
mIoU = 0.3796091079711914, median IoU: 0.37014952301979065, mAP = 0.02, mAP_50 = 0.091, mAP_75 = 0.002
TP: 863, FN: 0, FP: 270
Precision: 0.7616946160635482
Recall: 1.0
Validation evaluation:
mIoU = 0.31059837341308594, median IoU: 0.2716318368911743, mAP = 0.009, mAP_50 = 0.043, mAP_75 = 0.0
TP: 75, FN: 0, FP: 47
Precision: 0.6147540983606558
Recall: 1.0
Epoch time: 131.38335871696472
Estimated time remaining: 7:13:34

Epoch 2:
Train Loss: 0.8449456691741943, Total step: 21
Training evaluation:
mIoU = 0.475027859210968, median IoU: 0.481309711933136, mAP = 0.067, mAP_50 = 0.295, mAP_75 = 0.006
TP: 996, FN: 0, FP: 137
Precision: 0.8790820829655781
Recall: 1.0
Validation evaluation:
mIoU = 0.38866570591926575, median IoU: 0.3760471045970917, mAP = 0.032, mAP_50 = 0.172, mAP_75 = 0.002
TP: 93, FN: 0, FP: 29
Precision: 0.7622950819672131
Recall: 1.0
Epoch time: 125.01341342926025
Estimated tim

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 200
batch_size = 55
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

retinanet = retinanet_resnet50_fpn_v2(num_classes=2, detections_per_img =1, score_thresh = 0.05, nms_thresh = 0.5, min_size=min_size, max_size=max_size).to(device)

# Freeze all layers in the backbone
for param in retinanet.backbone.body.parameters():
    param.requires_grad = False

# Unfreeze the last two layers in the backbone
for param in retinanet.backbone.body.layer4.parameters():
    param.requires_grad = True
for param in retinanet.backbone.body.layer3.parameters():
    param.requires_grad = True
for param in retinanet.backbone.body.layer2.parameters():
    param.requires_grad = True

save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, GBUSV, 3 trainable layers"

params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, GBUSV, 3 trainable layers/checkpoints/checkpoint_epoch_125.pth", map_location=device)
retinanet.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
best_mAP = checkpoint['best_mAP']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler, last_epoch=last_epoch, best_mAP=best_mAP, train_scores=train_scores, val_scores=val_scores)

Best mAP so far: 0.27139872312545776

Epoch 126:
Train Loss: 0.16441434621810913, Total step: 21
Training evaluation:
mIoU = 0.877661406993866, median IoU: 0.8876245021820068, mAP = 0.761, mAP_50 = 0.99, mAP_75 = 0.955
TP: 1132, FN: 0, FP: 1
Precision: 0.999117387466902
Recall: 1.0
Validation evaluation:
mIoU = 0.6012313961982727, median IoU: 0.6501659750938416, mAP = 0.264, mAP_50 = 0.706, mAP_75 = 0.131
TP: 116, FN: 0, FP: 6
Precision: 0.9508196721311475
Recall: 1.0
Epoch time: 134.70419573783875
Estimated time remaining: 2:43:53

Epoch 127:
Train Loss: 0.1618228256702423, Total step: 21
Training evaluation:
mIoU = 0.879051148891449, median IoU: 0.8884961605072021, mAP = 0.764, mAP_50 = 0.99, mAP_75 = 0.965
TP: 1131, FN: 0, FP: 2
Precision: 0.9982347749338041
Recall: 1.0
Validation evaluation:
mIoU = 0.6039282083511353, median IoU: 0.6359359622001648, mAP = 0.263, mAP_50 = 0.698, mAP_75 = 0.125
TP: 116, FN: 0, FP: 6
Precision: 0.9508196721311475
Recall: 1.0
Epoch time: 125.0352962017

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 200
batch_size = 55
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

retinanet = retinanet_resnet50_fpn_v2(num_classes=2, detections_per_img =1, score_thresh = 0.05, nms_thresh = 0.5, min_size=min_size, max_size=max_size).to(device)

# Freeze all layers in the backbone
for param in retinanet.backbone.body.parameters():
    param.requires_grad = False

# Unfreeze the last two layers in the backbone
for param in retinanet.backbone.body.layer4.parameters():
    param.requires_grad = True
for param in retinanet.backbone.body.layer3.parameters():
    param.requires_grad = True
for param in retinanet.backbone.body.layer2.parameters():
    param.requires_grad = True

save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, GBUSV, 3 trainable layers"

params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, GBUSV, 3 trainable layers/checkpoints/checkpoint_epoch_170.pth", map_location=device)
retinanet.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
best_mAP = checkpoint['best_mAP']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler, last_epoch=last_epoch, best_mAP=best_mAP, train_scores=train_scores, val_scores=val_scores)

Best mAP so far: 0.27139872312545776

Epoch 171:
Train Loss: 0.13638347387313843, Total step: 21
Training evaluation:
mIoU = 0.8737919926643372, median IoU: 0.88197922706604, mAP = 0.753, mAP_50 = 0.99, mAP_75 = 0.964
TP: 1131, FN: 0, FP: 2
Precision: 0.9982347749338041
Recall: 1.0
Validation evaluation:
mIoU = 0.6029236316680908, median IoU: 0.6475881338119507, mAP = 0.26, mAP_50 = 0.702, mAP_75 = 0.123
TP: 114, FN: 0, FP: 8
Precision: 0.9344262295081968
Recall: 1.0
Epoch time: 146.35252904891968
Estimated time remaining: 1:8:18

Epoch 172:
Train Loss: 0.1422121524810791, Total step: 21
Training evaluation:
mIoU = 0.892672598361969, median IoU: 0.9039867520332336, mAP = 0.796, mAP_50 = 0.99, mAP_75 = 0.968
TP: 1132, FN: 0, FP: 1
Precision: 0.999117387466902
Recall: 1.0
Validation evaluation:
mIoU = 0.6028336882591248, median IoU: 0.6456969380378723, mAP = 0.268, mAP_50 = 0.717, mAP_75 = 0.147
TP: 115, FN: 0, FP: 7
Precision: 0.9426229508196722
Recall: 1.0
Epoch time: 126.5792734622955

### Retinanet with COCO weights

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 200
batch_size = 55

retinanet = retinanet_resnet50_fpn_v2(weights="COCO_V1", detections_per_img =1, score_thresh = 0.05, nms_thresh = 0.5, min_size=min_size, max_size=max_size).to(device)
num_classes=2
num_anchors = retinanet.head.classification_head.num_anchors
in_channels = retinanet.head.classification_head.cls_logits.in_channels
new_cls_logits = nn.Conv2d(in_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1).to(device)
torch.nn.init.normal_(new_cls_logits.weight, std=0.01)
torch.nn.init.constant_(new_cls_logits.bias, -math.log((1 - 0.01) / 0.01))
retinanet.head.classification_head.cls_logits = new_cls_logits
retinanet.head.classification_head.num_classes = num_classes

save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:09<00:00, 15.6MB/s]


Best mAP so far: 0

Epoch 1:
Train Loss: 1.5066062211990356, Total step: 21
Training evaluation:
mIoU = 0.0013798963045701385, median IoU: 0.0, mAP = 0.0, mAP_50 = 0.0, mAP_75 = 0.0
TP: 1, FN: 23, FP: 1109
Precision: 0.0009009009009009009
Recall: 0.041666666666666664
Validation evaluation:
mIoU = 0.001400450593791902, median IoU: 0.0, mAP = 0.0, mAP_50 = 0.0, mAP_75 = 0.0
TP: 0, FN: 2, FP: 120
Precision: 0.0
Recall: 0.0
Epoch time: 1061.53599858284
Estimated time remaining: 58:23:4

Epoch 2:
Train Loss: 1.349063754081726, Total step: 21
Training evaluation:
mIoU = 0.01220181118696928, median IoU: 0.0, mAP = 0.0, mAP_50 = 0.0, mAP_75 = 0.0
TP: 17, FN: 0, FP: 1116
Precision: 0.01500441306266549
Recall: 1.0
Validation evaluation:
mIoU = 0.009818969294428825, median IoU: 0.0, mAP = 0.0, mAP_50 = 0.0, mAP_75 = 0.0
TP: 0, FN: 0, FP: 122
Precision: 0.0
Recall: 0
Epoch time: 128.58360409736633
Estimated time remaining: 7:2:11

Epoch 3:
Train Loss: 1.131894826889038, Total step: 21
Training eva

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 100
batch_size = 55
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

retinanet = retinanet_resnet50_fpn_v2(weights="COCO_V1", detections_per_img =1, score_thresh = 0.05, nms_thresh = 0.5, min_size=min_size, max_size=max_size).to(device)
num_classes=2
num_anchors = retinanet.head.classification_head.num_anchors
in_channels = retinanet.head.classification_head.cls_logits.in_channels
new_cls_logits = nn.Conv2d(in_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1).to(device)
torch.nn.init.normal_(new_cls_logits.weight, std=0.01)
torch.nn.init.constant_(new_cls_logits.bias, -math.log((1 - 0.01) / 0.01))
retinanet.head.classification_head.cls_logits = new_cls_logits
retinanet.head.classification_head.num_classes = num_classes
params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_35.pth", map_location=device)
retinanet.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
best_mAP = checkpoint['best_mAP']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler, last_epoch=last_epoch, best_mAP=best_mAP, train_scores=train_scores, val_scores=val_scores)

Best mAP so far: 0.2574859857559204

Epoch 36:
Train Loss: 0.38868269324302673, Total step: 21
Training evaluation:
mIoU = 0.7326063513755798, median IoU: 0.7733424305915833, mAP = 0.46, mAP_50 = 0.886, mAP_75 = 0.433
TP: 1111, FN: 0, FP: 22
Precision: 0.9805825242718447
Recall: 1.0
Validation evaluation:
mIoU = 0.6106396913528442, median IoU: 0.6365611553192139, mAP = 0.256, mAP_50 = 0.663, mAP_75 = 0.157
TP: 112, FN: 0, FP: 10
Precision: 0.9180327868852459
Recall: 1.0
Epoch time: 415.83288502693176
Estimated time remaining: 7:16:37

Epoch 37:
Train Loss: 0.3840472996234894, Total step: 21
Training evaluation:
mIoU = 0.732841432094574, median IoU: 0.7746497988700867, mAP = 0.461, mAP_50 = 0.885, mAP_75 = 0.429
TP: 1106, FN: 0, FP: 27
Precision: 0.9761694616063548
Recall: 1.0
Validation evaluation:
mIoU = 0.6168451309204102, median IoU: 0.6585828065872192, mAP = 0.264, mAP_50 = 0.677, mAP_75 = 0.179
TP: 113, FN: 0, FP: 9
Precision: 0.9262295081967213
Recall: 1.0
Epoch time: 130.8210148

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 200
batch_size = 55
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

retinanet = retinanet_resnet50_fpn_v2(weights="COCO_V1", detections_per_img =1, score_thresh = 0.05, nms_thresh = 0.5, min_size=min_size, max_size=max_size).to(device)
num_classes=2
num_anchors = retinanet.head.classification_head.num_anchors
in_channels = retinanet.head.classification_head.cls_logits.in_channels
new_cls_logits = nn.Conv2d(in_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1).to(device)
torch.nn.init.normal_(new_cls_logits.weight, std=0.01)
torch.nn.init.constant_(new_cls_logits.bias, -math.log((1 - 0.01) / 0.01))
retinanet.head.classification_head.cls_logits = new_cls_logits
retinanet.head.classification_head.num_classes = num_classes
params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_70.pth", map_location=device)
retinanet.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
best_mAP = checkpoint['best_mAP']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler, last_epoch=last_epoch, best_mAP=best_mAP, train_scores=train_scores, val_scores=val_scores)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 149MB/s]


Best mAP so far: 0.30000853538513184

Epoch 71:
Train Loss: 0.3137096166610718, Total step: 21
Training evaluation:
mIoU = 0.7728212475776672, median IoU: 0.7981986999511719, mAP = 0.535, mAP_50 = 0.956, mAP_75 = 0.537
TP: 1122, FN: 0, FP: 11
Precision: 0.9902912621359223
Recall: 1.0
Validation evaluation:
mIoU = 0.6511924266815186, median IoU: 0.6926107406616211, mAP = 0.309, mAP_50 = 0.794, mAP_75 = 0.191
TP: 116, FN: 0, FP: 6
Precision: 0.9508196721311475
Recall: 1.0
Epoch time: 676.609854221344
Estimated time remaining: 24:3:26

Epoch 72:
Train Loss: 0.313943475484848, Total step: 21
Training evaluation:
mIoU = 0.7706896066665649, median IoU: 0.7883628010749817, mAP = 0.524, mAP_50 = 0.956, mAP_75 = 0.529
TP: 1119, FN: 0, FP: 14
Precision: 0.9876434245366285
Recall: 1.0
Validation evaluation:
mIoU = 0.6534900069236755, median IoU: 0.6991515159606934, mAP = 0.306, mAP_50 = 0.779, mAP_75 = 0.177
TP: 117, FN: 0, FP: 5
Precision: 0.9590163934426229
Recall: 1.0
Epoch time: 134.428953409

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 200
batch_size = 55
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

retinanet = retinanet_resnet50_fpn_v2(weights="COCO_V1", detections_per_img =1, score_thresh = 0.05, nms_thresh = 0.5, min_size=min_size, max_size=max_size).to(device)
num_classes=2
num_anchors = retinanet.head.classification_head.num_anchors
in_channels = retinanet.head.classification_head.cls_logits.in_channels
new_cls_logits = nn.Conv2d(in_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1).to(device)
torch.nn.init.normal_(new_cls_logits.weight, std=0.01)
torch.nn.init.constant_(new_cls_logits.bias, -math.log((1 - 0.01) / 0.01))
retinanet.head.classification_head.cls_logits = new_cls_logits
retinanet.head.classification_head.num_classes = num_classes
params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_100.pth", map_location=device)
retinanet.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
best_mAP = checkpoint['best_mAP']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler, last_epoch=last_epoch, best_mAP=best_mAP, train_scores=train_scores, val_scores=val_scores)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 88.9MB/s]


Best mAP so far: 0.31726309657096863

Epoch 101:
Train Loss: 0.2783876657485962, Total step: 21
Training evaluation:
mIoU = 0.7916655540466309, median IoU: 0.8088105916976929, mAP = 0.575, mAP_50 = 0.979, mAP_75 = 0.624
TP: 1124, FN: 0, FP: 9
Precision: 0.9920564872021183
Recall: 1.0
Validation evaluation:
mIoU = 0.6615225076675415, median IoU: 0.7046943306922913, mAP = 0.309, mAP_50 = 0.812, mAP_75 = 0.188
TP: 118, FN: 0, FP: 4
Precision: 0.9672131147540983
Recall: 1.0
Epoch time: 575.6993367671967
Estimated time remaining: 15:40:19

Epoch 102:
Train Loss: 0.2805318236351013, Total step: 21
Training evaluation:
mIoU = 0.7927755117416382, median IoU: 0.809079647064209, mAP = 0.572, mAP_50 = 0.978, mAP_75 = 0.623
TP: 1127, FN: 0, FP: 6
Precision: 0.9947043248014121
Recall: 1.0
Validation evaluation:
mIoU = 0.6582143902778625, median IoU: 0.6963031888008118, mAP = 0.303, mAP_50 = 0.798, mAP_75 = 0.164
TP: 117, FN: 0, FP: 5
Precision: 0.9590163934426229
Recall: 1.0
Epoch time: 137.6148149

KeyboardInterrupt: 

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 200
batch_size = 55
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

retinanet = retinanet_resnet50_fpn_v2(weights="COCO_V1", detections_per_img=1, score_thresh=0.05, nms_thresh=0.5, min_size=min_size, max_size=max_size).to(device)
num_classes=2
num_anchors = retinanet.head.classification_head.num_anchors
in_channels = retinanet.head.classification_head.cls_logits.in_channels
new_cls_logits = nn.Conv2d(in_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1).to(device)
torch.nn.init.normal_(new_cls_logits.weight, std=0.01)
torch.nn.init.constant_(new_cls_logits.bias, -math.log((1 - 0.01) / 0.01))
retinanet.head.classification_head.cls_logits = new_cls_logits
retinanet.head.classification_head.num_classes = num_classes
params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_155.pth", map_location=device)
retinanet.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
best_mAP = checkpoint['best_mAP']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler, last_epoch=last_epoch, best_mAP=best_mAP, train_scores=train_scores, val_scores=val_scores)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 132MB/s]


Best mAP so far: 0.31726309657096863

Epoch 156:
Train Loss: 0.23180674016475677, Total step: 21
Training evaluation:
mIoU = 0.8216488361358643, median IoU: 0.83348548412323, mAP = 0.631, mAP_50 = 0.99, mAP_75 = 0.77
TP: 1129, FN: 0, FP: 4
Precision: 0.9964695498676082
Recall: 1.0
Validation evaluation:
mIoU = 0.6543325185775757, median IoU: 0.702592670917511, mAP = 0.307, mAP_50 = 0.777, mAP_75 = 0.161
TP: 119, FN: 0, FP: 3
Precision: 0.9754098360655737
Recall: 1.0
Epoch time: 594.8875269889832
Estimated time remaining: 7:6:20

Epoch 157:
Train Loss: 0.2356361746788025, Total step: 21
Training evaluation:
mIoU = 0.819092333316803, median IoU: 0.8340110182762146, mAP = 0.628, mAP_50 = 0.99, mAP_75 = 0.742
TP: 1129, FN: 0, FP: 4
Precision: 0.9964695498676082
Recall: 1.0
Validation evaluation:
mIoU = 0.654219925403595, median IoU: 0.6931415796279907, mAP = 0.301, mAP_50 = 0.769, mAP_75 = 0.139
TP: 119, FN: 0, FP: 3
Precision: 0.9754098360655737
Recall: 1.0
Epoch time: 136.90944385528564


In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
num_epochs = 200
batch_size = 55
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

retinanet = retinanet_resnet50_fpn_v2(weights="COCO_V1", detections_per_img=1, score_thresh=0.05, nms_thresh=0.5, min_size=min_size, max_size=max_size).to(device)
num_classes=2
num_anchors = retinanet.head.classification_head.num_anchors
in_channels = retinanet.head.classification_head.cls_logits.in_channels
new_cls_logits = nn.Conv2d(in_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1).to(device)
torch.nn.init.normal_(new_cls_logits.weight, std=0.01)
torch.nn.init.constant_(new_cls_logits.bias, -math.log((1 - 0.01) / 0.01))
retinanet.head.classification_head.cls_logits = new_cls_logits
retinanet.head.classification_head.num_classes = num_classes
params = [p for p in retinanet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_180.pth", map_location=device)
retinanet.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
best_mAP = checkpoint['best_mAP']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(retinanet, save_model_dir=save_model_dir, num_epochs=num_epochs, optimizer=optimizer, train_transform=train_transform, eval_transform=eval_transform,
            batch_size=batch_size, patience=None, checkpoint_save_rate=5, lr_scheduler=lr_scheduler, last_epoch=last_epoch, best_mAP=best_mAP, train_scores=train_scores, val_scores=val_scores)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 98.8MB/s]


Best mAP so far: 0.31726309657096863

Epoch 181:
Train Loss: 0.21788965165615082, Total step: 21
Training evaluation:
mIoU = 0.8294175267219543, median IoU: 0.8409611582756042, mAP = 0.651, mAP_50 = 0.99, mAP_75 = 0.799
TP: 1130, FN: 0, FP: 3
Precision: 0.9973521624007061
Recall: 1.0
Validation evaluation:
mIoU = 0.6526966094970703, median IoU: 0.6890166997909546, mAP = 0.305, mAP_50 = 0.762, mAP_75 = 0.155
TP: 118, FN: 0, FP: 4
Precision: 0.9672131147540983
Recall: 1.0
Epoch time: 480.66511154174805
Estimated time remaining: 2:24:12

Epoch 182:
Train Loss: 0.21744279563426971, Total step: 21
Training evaluation:
mIoU = 0.8294535875320435, median IoU: 0.8423455953598022, mAP = 0.65, mAP_50 = 0.99, mAP_75 = 0.814
TP: 1130, FN: 0, FP: 3
Precision: 0.9973521624007061
Recall: 1.0
Validation evaluation:
mIoU = 0.6549651026725769, median IoU: 0.6939677000045776, mAP = 0.31, mAP_50 = 0.762, mAP_75 = 0.159
TP: 118, FN: 0, FP: 4
Precision: 0.9672131147540983
Recall: 1.0
Epoch time: 144.86498880

## Inference

In [ ]:
width = 224
height = 224
min_size = 224
max_size = 224
batch_size=70
eval_transform = get_eval_transforms(width, height)

retinanet = retinanet_resnet50_fpn_v2(weights="COCO_V1", detections_per_img =1, score_thresh = 0.05, nms_thresh = 0.5, min_size=min_size, max_size=max_size).to(device)
num_classes=2
num_anchors = retinanet.head.classification_head.num_anchors
in_channels = retinanet.head.classification_head.cls_logits.in_channels
new_cls_logits = nn.Conv2d(in_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1).to(device)
torch.nn.init.normal_(new_cls_logits.weight, std=0.01)
torch.nn.init.constant_(new_cls_logits.bias, -math.log((1 - 0.01) / 0.01))
retinanet.head.classification_head.cls_logits = new_cls_logits
retinanet.head.classification_head.num_classes = num_classes
checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/Retinanet with COCO weights/checkpoints/best_model_epoch_91.pth", map_location=device)
retinanet.load_state_dict(checkpoint['model_state_dict'])

val_data = InputDataset(imgs_folder_path, val_txt_path, gold_file_path, img_transforms=eval_transform)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

_, predicted_boxes = evaluate_model(retinanet, val_loader, width=width, height=height, return_bboxes=True)

In [ ]:
with open("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/Retinanet with COCO weights/Retinanet_COCO_pred_boxes.json", "w") as f:
  json.dump(predicted_boxes,f)

# GBCNet

## Evaluation

In [ ]:
def evaluate_model(model, dataloader):
  model.eval()
  y_pred = []
  y_logits = []
  y_true = []
  fnames = []
  with torch.inference_mode():
    for i, data in enumerate(dataloader, 0):
      images_ids, inputs, targets = data[0],\
                                    torch.stack([image.to(device) for image in data[1]]),\
                                    [{k: v.to(device) if torch.is_tensor(v) else v for k, v in t.items()} for t in data[2]]

      cls_logits = model(inputs)
      _, cls_pred = torch.max(cls_logits, dim=1)

      y_true += [x['true_labels'].item() for x in targets]
      y_pred += cls_pred.tolist()
      #y_logits += cls_logits.tolist()
      fnames += images_ids
  #y_prop_pred = [F.softmax(torch.tensor(x), dim=1) for x in y_logits]
  acc = accuracy_score(y_true, y_pred)
  cfm = confusion_matrix(y_true, y_pred)
  spec = (cfm[0][0] + cfm[0][1] + cfm[1][0] + cfm[1][1])/(np.sum(cfm[0]) + np.sum(cfm[1]))
  sens = cfm[2][2]/np.sum(cfm[2])
  #AUC = roc_auc_score(y_true, y_prop_pred, multi_class='ovr')
  print('Acc: {:.4f} Spec: {:.4f} Sens: {:.4f}'.format(acc, spec, sens))

  return acc, cfm, y_true, y_pred, fnames

## Training

In [ ]:
def train_model(model, criterion, num_epochs, optimizer, va_curriculum, batch_size, save_model_dir, train_transform, eval_transform,
                predicted_bboxes_json, best_acc=0, num_worse_epochs=0, patience=None, checkpoint_save_rate=5, last_epoch=0,
                train_scores=[], val_scores=[], lr_scheduler=None):

  # Validation DataLoader
  val_dataset = InputDataset(imgs_folder_path, val_txt_path, gold_file_path, predicted_bboxes_json=predicted_bboxes_json,
                             do_crop="pred_boxes", to_blur=False, img_transforms=eval_transform)
  val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn= collate_fn)

  best_epoch = 0
  best_model = None
  show_num_images = True

  os.makedirs(f'{save_model_dir}/checkpoints', exist_ok=True)
  if patience is not None:
    os.makedirs(f'{save_model_dir}/Final Models', exist_ok=True)

  print(f"Last Epoch: {last_epoch}, Start Epoch: {last_epoch+1}, End Epoch: {num_epochs}")
  print(f"VA Curriculum: {va_curriculum}")
  print(f"Train Scores Length: {len(train_scores)}, Val Scores Length: {len(val_scores)}")
  print(f"Best accuracy so far: {best_acc}")
  for epoch in range(last_epoch+1, num_epochs+1):

    print(f'\nEpoch {epoch}:')
    start_time = time.time()

    if va_curriculum:
      if epoch <= 10:
        train_dataset = InputDataset(imgs_folder_path, train_txt_path, gold_file_path, do_crop="gold_boxes", to_blur=True, blur_kernel_size=(65,65), sigma=16, img_transforms=train_transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn= collate_fn)
      elif epoch >10 and epoch <=15:
        train_dataset = InputDataset(imgs_folder_path, train_txt_path, gold_file_path, do_crop="gold_boxes", to_blur=True, blur_kernel_size=(33,33), sigma=8, img_transforms=train_transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn= collate_fn)
      elif epoch >15 and epoch <=20:
        train_dataset = InputDataset(imgs_folder_path, train_txt_path, gold_file_path, do_crop="gold_boxes", to_blur=True, blur_kernel_size=(17,17), sigma=4, img_transforms=train_transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn= collate_fn)
      elif epoch >20 and epoch <=25:
        train_dataset = InputDataset(imgs_folder_path, train_txt_path, gold_file_path, do_crop="gold_boxes", to_blur=True, blur_kernel_size=(9,9), sigma=2, img_transforms=train_transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn= collate_fn)
      elif epoch >25 and epoch <=30:
        train_dataset = InputDataset(imgs_folder_path, train_txt_path, gold_file_path, do_crop="gold_boxes", to_blur=True, blur_kernel_size=(5,5), sigma=1, img_transforms=train_transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn= collate_fn)
      else:
        train_dataset = InputDataset(imgs_folder_path, train_txt_path, gold_file_path, do_crop="gold_boxes", to_blur=False, img_transforms= train_transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn= collate_fn)
    else:
      train_dataset = InputDataset(imgs_folder_path, train_txt_path, gold_file_path, do_crop="gold_boxes", to_blur=False, img_transforms=train_transform)
      train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn= collate_fn)

    if show_num_images == True:
      print(f"Number of Train images: {len(train_dataset)}")
      print(f"Number of Test images: {len(val_dataset)}")
      show_num_images = False

    # Training Loop
    model.train()
    running_loss = 0.0
    total_step = len(train_loader)
    for i, data in enumerate(train_loader, 0):
      images_ids, inputs, targets = data[0],\
                                    torch.stack([image.to(device) for image in data[1]]),\
                                    [{k: v.to(device) if torch.is_tensor(v) else v for k, v in t.items()} for t in data[2]]

      optimizer.zero_grad()

      cls_logits = model(inputs)
      true_labels = torch.stack([x['true_labels'] for x in targets])
      loss = criterion(cls_logits.to(device), true_labels.to(device))

      loss.backward()

      optimizer.step()

      running_loss += loss.item()

    train_loss = running_loss/total_step
    print(f"Train Loss: {train_loss}, Total step: {total_step}")


    # Evaluating Model
    model.eval()
    print("Train Evaluation:")
    train_acc, train_cfm, train_y_true, train_y_pred, train_fnames = evaluate_model(model, train_loader)
    train_scores.append(train_acc)

    print("Val Evaluation:")
    val_acc, val_cfm, val_y_true, val_y_pred, val_fnames = evaluate_model(model, val_loader)
    val_scores.append(val_acc)

    if (lr_scheduler is not None) and (epoch>=50):
      lr_scheduler.step(val_acc)

    # Early Stop
    if val_acc > best_acc:
      best_acc = val_acc
      best_epoch = epoch
      best_model = copy.deepcopy(model)
      num_worse_epochs = 0
    else:
      num_worse_epochs += 1
      if patience is not None:
        if num_worse_epochs >= patience:
          # save the best model and its optimizer
          if best_model is not None:
            torch.save({'model_state_dict': best_model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'best_acc': best_acc,
                        'epoch': best_epoch},
                        f'{save_model_dir}/Final Models/best_model_epoch_{best_epoch}.pth')
            print(f'Best model is saved to: {save_model_dir}/Final Models/best_model_epoch_{best_epoch}.pth')

          else:
            print("\nNo model to save!")

          # save the last model and its optimizer
          torch.save({'model_state_dict': model.state_dict(),
                      'optimizer_state_dict': optimizer.state_dict(),
                      #'lr_scheduler_state_dict': lr_scheduler.state_dict(),
                      'best_acc': best_acc,
                      'epoch': epoch,
                      'num_worse_epochs': num_worse_epochs,
                      'train_scores': train_scores,
                      'val_scores': val_scores},
                      f'{save_model_dir}/Final Models/last_model_epoch_{epoch}.pth')
          print(f'Last model is saved to: {save_model_dir}/Final Models/last_model_epoch_{epoch}.pth')

          print(f"No better results for last {patience} epochs!\nBest model was for epoch {best_epoch} with best mAP: {best_acc}")
          break


    if (epoch) % checkpoint_save_rate == 0:
      random_state = random.getstate()
      np_random_state = np.random.get_state()
      torch_random_state = torch.get_rng_state()
      torch.save({'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'best_acc': best_acc,
                  'train_scores': train_scores,
                  'val_scores': val_scores,
                  'epoch': epoch,
                  'num_worse_epochs': num_worse_epochs,
                  'random_state': random_state,
                  'np_random_state': np_random_state,
                  'torch_random_state': torch_random_state},
                  f'{save_model_dir}/checkpoints/checkpoint_epoch_{epoch}.pth')
      print(f"\nCheckpoint is saved to: {save_model_dir}/checkpoints/checkpoint_epoch_{epoch}.pth\n")

      if best_model is not None:
        torch.save({'model_state_dict': best_model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'best_acc': best_acc,
                    'epoch': best_epoch},
                    f'{save_model_dir}/checkpoints/best_model_epoch_{best_epoch}.pth')
        print(f'Best model is saved to: {save_model_dir}/checkpoints/best_model_epoch_{best_epoch}.pth')
        print(f"Best model was for epoch {best_epoch} with best Acc: {best_acc}")

      else:
        print("\nNo Best model to save!")


    # Time
    end_time = time.time()
    elapsed_time = end_time - start_time
    remaining_epochs = num_epochs - epoch
    remaining_time = round(elapsed_time * remaining_epochs)
    hours, remainder = divmod(remaining_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f"Estimated time remaining: {hours}:{minutes}:{seconds}")

    print(f"GPU Memory Used: {torch.cuda.memory_allocated()}")

## Experiments

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = True
num_epochs = 200
batch_size = 55
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
predicted_bboxes_json = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/Retinanet with COCO weights/Retinanet_COCO_pred_boxes.json"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCNet"
criterion = nn.CrossEntropyLoss()

gbcnet = GbcNet(num_cls=3, pretrain= False, att_mode= "1")
gbcnet.load_state_dict(torch.load(gbcnet_weight_path, map_location = device))
gbcnet.net = gbcnet.net.float().to(device)

params = [p for p in gbcnet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcnet, criterion=criterion, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform,
            eval_transform=eval_transform, predicted_bboxes_json=predicted_bboxes_json, checkpoint_save_rate=5,
            va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir, lr_scheduler=lr_scheduler)

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = True
num_epochs = 200
batch_size = 55
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
predicted_bboxes_json = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/Retinanet_COCO_pred_boxes.json"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCNet"
criterion = nn.CrossEntropyLoss()
gbcnet = GbcNet(num_cls=3, pretrain= False, att_mode= "1")
gbcnet.net = gbcnet.net.float().to(device)
params = [p for p in gbcnet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCNet/checkpoints/checkpoint_epoch_15.pth", map_location=device)
gbcnet.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
best_acc = checkpoint['best_acc']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcnet, criterion=criterion, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform,
            eval_transform=eval_transform, predicted_bboxes_json=predicted_bboxes_json, checkpoint_save_rate=5,
            va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir, lr_scheduler=lr_scheduler,
            last_epoch=last_epoch, best_acc=best_acc, train_scores=train_scores, val_scores=val_scores)

Last Epoch: 15, Start Epoch: 16, End Epoch: 200
VA Curriculum: True
Train Scores Length: 15, Val Scores Length: 15
Best accuracy so far: 0.6557377049180327

Epoch 16:
Number of Train images: 1133
Number of Test images: 122
Train Loss: 0.3140141985246113, Total step: 21
Train Evaluation:
Acc: 0.9020 Spec: 0.9824 Sens: 0.8969
Val Evaluation:
Acc: 0.7869 Spec: 0.8250 Sens: 0.8810
Estimated time remaining: 31:7:29
GPU Memory Used: 595699200

Epoch 17:
Train Loss: 0.32123562196890515, Total step: 21
Train Evaluation:
Acc: 0.8906 Spec: 0.9802 Sens: 0.9013
Val Evaluation:
Acc: 0.8197 Spec: 0.8375 Sens: 0.8810
Estimated time remaining: 7:43:41
GPU Memory Used: 596079104

Epoch 18:
Train Loss: 0.3399037896167664, Total step: 21
Train Evaluation:
Acc: 0.9100 Spec: 0.9890 Sens: 0.8969
Val Evaluation:
Acc: 0.8443 Spec: 0.8750 Sens: 0.8810
Estimated time remaining: 7:40:10
GPU Memory Used: 599726592

Epoch 19:
Train Loss: 0.28864200768016635, Total step: 21
Train Evaluation:
Acc: 0.9020 Spec: 0.987

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = True
num_epochs = 200
batch_size = 55
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
predicted_bboxes_json = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/Retinanet_COCO_pred_boxes.json"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCNet"
criterion = nn.CrossEntropyLoss()
gbcnet = GbcNet(num_cls=3, pretrain= False, att_mode= "1")
gbcnet.net = gbcnet.net.float().to(device)
params = [p for p in gbcnet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCNet/checkpoints/checkpoint_epoch_45.pth", map_location=device)
gbcnet.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
best_acc = checkpoint['best_acc']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcnet, criterion=criterion, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform,
            eval_transform=eval_transform, predicted_bboxes_json=predicted_bboxes_json, checkpoint_save_rate=5,
            va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir, lr_scheduler=lr_scheduler,
            last_epoch=last_epoch, best_acc=best_acc, train_scores=train_scores, val_scores=val_scores)

Last Epoch: 45, Start Epoch: 46, End Epoch: 200
VA Curriculum: True
Train Scores Length: 45, Val Scores Length: 45
Best accuracy so far: 0.8770491803278688

Epoch 46:
Number of Train images: 1133
Number of Test images: 122
Train Loss: 0.2671441308089665, Total step: 21
Train Evaluation:
Acc: 0.9232 Spec: 0.9846 Sens: 0.9283
Val Evaluation:
Acc: 0.8525 Spec: 0.9500 Sens: 0.7143
Estimated time remaining: 42:15:1
GPU Memory Used: 486109184

Epoch 47:
Train Loss: 0.2832397868235906, Total step: 21
Train Evaluation:
Acc: 0.9126 Spec: 0.9901 Sens: 0.8834
Val Evaluation:
Acc: 0.8689 Spec: 0.9625 Sens: 0.7143
Estimated time remaining: 5:39:20
GPU Memory Used: 486109184

Epoch 48:
Train Loss: 0.25107336079790477, Total step: 21
Train Evaluation:
Acc: 0.9197 Spec: 0.9813 Sens: 0.9103
Val Evaluation:
Acc: 0.8607 Spec: 0.9500 Sens: 0.7381
Estimated time remaining: 5:37:32
GPU Memory Used: 486109184

Epoch 49:
Train Loss: 0.23924889380023592, Total step: 21
Train Evaluation:
Acc: 0.9144 Spec: 0.987

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = True
num_epochs = 200
batch_size = 55
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
predicted_bboxes_json = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/Retinanet_COCO_pred_boxes.json"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCNet"
criterion = nn.CrossEntropyLoss()
gbcnet = GbcNet(num_cls=3, pretrain= False, att_mode= "1")
gbcnet.net = gbcnet.net.float().to(device)
params = [p for p in gbcnet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCNet/checkpoints/checkpoint_epoch_90.pth", map_location=device)
gbcnet.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
best_acc = checkpoint['best_acc']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcnet, criterion=criterion, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform,
            eval_transform=eval_transform, predicted_bboxes_json=predicted_bboxes_json, checkpoint_save_rate=5,
            va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir, lr_scheduler=lr_scheduler,
            last_epoch=last_epoch, best_acc=best_acc, train_scores=train_scores, val_scores=val_scores)

Last Epoch: 90, Start Epoch: 91, End Epoch: 200
VA Curriculum: True
Train Scores Length: 90, Val Scores Length: 90
Best accuracy so far: 0.8852459016393442

Epoch 91:
Number of Train images: 1133
Number of Test images: 122
Train Loss: 0.21279944408507573, Total step: 21
Train Evaluation:
Acc: 0.9417 Spec: 0.9912 Sens: 0.9507
Val Evaluation:
Acc: 0.8770 Spec: 0.9500 Sens: 0.7857
Estimated time remaining: 32:3:9
GPU Memory Used: 486109184

Epoch 92:
Train Loss: 0.19412613766533987, Total step: 21
Train Evaluation:
Acc: 0.9523 Spec: 0.9945 Sens: 0.9283
Val Evaluation:
Acc: 0.8689 Spec: 0.9500 Sens: 0.7619
Estimated time remaining: 3:53:9
GPU Memory Used: 486109184

Epoch 93:
Train Loss: 0.17832490908248083, Total step: 21
Train Evaluation:
Acc: 0.9488 Spec: 0.9989 Sens: 0.9193
Val Evaluation:
Acc: 0.8689 Spec: 0.9625 Sens: 0.7381
Estimated time remaining: 3:54:0
GPU Memory Used: 486109184

Epoch 94:
Train Loss: 0.20295888043585278, Total step: 21
Train Evaluation:
Acc: 0.9479 Spec: 0.9912

KeyboardInterrupt: 

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = True
num_epochs = 200
batch_size = 55
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
predicted_bboxes_json = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/Retinanet_COCO_pred_boxes.json"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCNet"
criterion = nn.CrossEntropyLoss()
gbcnet = GbcNet(num_cls=3, pretrain= False, att_mode= "1")
gbcnet.net = gbcnet.net.float().to(device)
params = [p for p in gbcnet.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCNet/checkpoints/checkpoint_epoch_170.pth", map_location=device)
gbcnet.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
best_acc = checkpoint['best_acc']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcnet, criterion=criterion, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform,
            eval_transform=eval_transform, predicted_bboxes_json=predicted_bboxes_json, checkpoint_save_rate=5,
            va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir, lr_scheduler=lr_scheduler,
            last_epoch=last_epoch, best_acc=best_acc, train_scores=train_scores, val_scores=val_scores)

Last Epoch: 170, Start Epoch: 171, End Epoch: 200
VA Curriculum: True
Train Scores Length: 170, Val Scores Length: 170
Best accuracy so far: 0.8852459016393442

Epoch 171:
Number of Train images: 1133
Number of Test images: 122
Train Loss: 0.14197639686365923, Total step: 21
Train Evaluation:
Acc: 0.9691 Spec: 0.9989 Sens: 0.9776
Val Evaluation:
Acc: 0.8689 Spec: 0.9500 Sens: 0.7619
Estimated time remaining: 1:6:47
GPU Memory Used: 486109184

Epoch 172:
Train Loss: 0.13134333739678064, Total step: 21
Train Evaluation:
Acc: 0.9700 Spec: 0.9978 Sens: 0.9462
Val Evaluation:
Acc: 0.8770 Spec: 0.9625 Sens: 0.7619
Estimated time remaining: 0:55:47
GPU Memory Used: 486109184

Epoch 173:
Train Loss: 0.11449916536609332, Total step: 21
Train Evaluation:
Acc: 0.9691 Spec: 0.9934 Sens: 0.9641
Val Evaluation:
Acc: 0.8689 Spec: 0.9500 Sens: 0.7619
Estimated time remaining: 0:53:15
GPU Memory Used: 486109184

Epoch 174:
Train Loss: 0.1381497092190243, Total step: 21
Train Evaluation:
Acc: 0.9673 Spe

# GBCRet

## Evaluation

In [ ]:
def evaluate_model(model, dataloader, return_bboxes=False):
  model.eval()
  all_IoUs = []
  mAP_metric = MeanAveragePrecision(box_format='xyxy', iou_type='bbox')
  y_pred = []
  y_true = []
  fnames = []
  TPs = 0
  FNs = 0
  FPs = 0
  precision = 0
  recall = 0
  all_bboxes= {}
  with torch.inference_mode():
    for i, data in enumerate(dataloader, 0):
      images_ids, inputs, targets = data[0],\
                                    [image.to(device) for image in data[1]],\
                                    [{k: v.to(device) if torch.is_tensor(v) else v for k, v in t.items()} for t in data[2]]

      pred_cls, pred_bboxes = model(inputs, targets)

      y_true += [x['true_labels'].item() for x in targets]
      y_pred += pred_cls.tolist()
      fnames += images_ids

      IoUs = evaluate_IoUs(targets, pred_bboxes)
      all_IoUs += IoUs

      tp, fp, fn = compute_bbox_metrics(targets, pred_bboxes)
      TPs += tp
      FPs += fp
      FNs += fn

      mAP_metric.update(pred_bboxes, targets)

      if return_bboxes:
        for i in range(len(targets)):
          fname = images_ids[i]
          target_bbox = targets[i]['boxes']
          pred_bbox = pred_bboxes[i]['boxes']
          all_bboxes[fname] = {"Boxes": pred_bbox, "Gold": target_bbox}

  mIoU = torch.stack(all_IoUs).mean()
  medIoU = torch.stack(all_IoUs).median()
  mAP_dict = mAP_metric.compute()
  mAP = mAP_dict['map']
  print(f"mIoU = {mIoU}, median IoU: {medIoU}, mAP = {round(mAP_dict['map'].item(),3)}, mAP_50 = {round(mAP_dict['map_50'].item(),3)}, mAP_75 = {round(mAP_dict['map_75'].item(),3)}")

  if (TPs + FPs) != 0:
    precision = TPs / (TPs + FPs)
  if (TPs + FNs) != 0:
    recall = TPs / (TPs + FNs)
  print(f"TP: {TPs}, FN: {FNs}, FP: {FPs}")
  print("Precision: {}\nRecall: {}".format(precision, recall))

  acc = accuracy_score(y_true, y_pred)
  cfm = confusion_matrix(y_true, y_pred)
  spec = (cfm[0][0] + cfm[0][1] + cfm[1][0] + cfm[1][1])/(np.sum(cfm[0]) + np.sum(cfm[1]))
  sens = cfm[2][2]/np.sum(cfm[2])
  n_nn_acc = (cfm[0,0] + np.sum(cfm[1:,1:]))/np.sum(cfm)
  b_m_acc = (cfm[1,1] + cfm[2,2])/np.sum(cfm[1:,1:])
  print('Acc: {:.4f} Spec: {:.4f} Sens: {:.4f}\nNormal vs Not Noraml Acc: {:.4f} Benign vs Malignant Acc: {:.4f}'.format(acc, spec, sens, n_nn_acc, b_m_acc))

  return acc, round(mAP_dict['map'].item(),3), cfm, y_true, y_pred, fnames, all_bboxes

## Model

### Attention

In [ ]:
class FeatureMapAttention(nn.Module):
  def __init__(self, embed_dim, num_heads):
    super().__init__()
    self.multiheadattn = nn.MultiheadAttention(embed_dim, num_heads, batch_first=True)

  def forward(self, feature_map):
    # feature map shape: [batch_size, 256, H, W]
    # Reshape to: [batch_size, H*W, 256]
    x = feature_map.reshape(feature_map.size(0), -1, 256)
    # Apply Attention
    attn_output, _ = self.multiheadattn(x,x,x)
    # Reshape back to original shape: [batch_size, 256, H, W]
    return attn_output.reshape(*feature_map.shape)

class VectorAttention(nn.Module):
  def __init__(self, embed_dim, num_heads):
    super().__init__()
    self.multiheadattn = nn.MultiheadAttention(embed_dim, num_heads, batch_first=True)

  def forward(self, x):
    # x shape: [batch_size, 256 + 2048]
    # Reshape to: [batch_size, 1, 256 + 2048]
    x = x.unsqueeze(1)
    # Apply Attention
    attn_output, _ = self.multiheadattn(x, x, x)
    # Remove the extra dimension: [batch_size, 256 + 2048]
    return attn_output.squeeze(1)


### Main model

In [ ]:
class GBCRet_combined(nn.Module):
  def __init__(self, combined="None", continue_training=False, criterion=None, gbcnet_weight_path=None, retinanet_weight_path=None,
               do_hierarchical_cls=False, retinanet_backbone="resnet50-coco", gold_curriculum=False, p=0.15, width=224, height=224,
               attn_num_heads=4, no_attn=True, train_gold_bbox=True, min_size=800, max_size=1333, dropout=0.4, detections_per_img=1,
               regression_loss=None, rn_trainable_layers=None, gbcnet_va_curriculum=False, load_pretrained_retinanet=False,
               loss_weights = {"n_nn_loss_weight": 1.0, "b_m_loss_weight": 1.0, "classification_loss_weight": 1.0, "retinanet_loss_weight": 1.0},
               nnn_fc_layers="1", bm_fc_layers="1"):
    super(GBCRet_combined, self).__init__()
    # Settings
    self.loss_weights=loss_weights
    self.load_pretrained_retinanet=load_pretrained_retinanet
    self.gbcnet_va_curriculum=gbcnet_va_curriculum
    self.rn_trainable_layers=rn_trainable_layers
    self.retinanet_backbone=retinanet_backbone
    self.combined=combined
    self.no_attn = no_attn
    self.do_hierarchical_cls=do_hierarchical_cls
    self.continue_training=continue_training
    self.detections_per_img=detections_per_img
    self.p = p
    self.width = width
    self.height = height
    self.min_size = min_size
    self.max_size = max_size
    self.criterion = criterion
    self.train_gold_bbox = train_gold_bbox
    self.gold_curriculum = gold_curriculum
    self.regression_loss = regression_loss
    self.current_epoch = -1
    if isinstance(dropout, dict):
      n_nn_dropout=dropout['n_nn_dropout']
      b_m_dropout=dropout['b_m_dropout']
      dropout=0
      print(f"n_nn_dropout: {n_nn_dropout}, b_m_dropout: {b_m_dropout}")
    else:
      n_nn_dropout=dropout
      b_m_dropout=dropout
    # Status Flags
    self.gbcnet_va_curriculum_flag=False
    self.combined_status_flag=False
    self.no_attn_status_flag=False
    self.train_gold_box_status_flag=False
    self.gold_curriculum_status_flag=False
    self.num_gold_boxes_status_flag=False
    self.hierarchical_classification_status_flag=False
    # Loading models
    self.retinanet = self.load_retinanet(retinanet_weight_path)
    self.gbcnet = self.load_gbcnet(gbcnet_weight_path)
    # Hooks
    self.retinanet_features = None
    self.gbcnet_features = None
    self.resnet_layer1_features = None
    self.resnet_layer2_features = None
    self.resnet_layer3_features = None
    self.resnet_layer4_features = None
    self.gbcnet.net.layer4.register_forward_hook(self._get_gbcnet_features_hook)
    self.retinanet.backbone.register_forward_hook(self._get_retinanet_features_hook)
    self.retinanet.backbone.body.layer1.register_forward_hook(self._get_resnet_layer1_features_hook)
    self.retinanet.backbone.body.layer2.register_forward_hook(self._get_resnet_layer2_features_hook)
    self.retinanet.backbone.body.layer3.register_forward_hook(self._get_resnet_layer3_features_hook)
    self.retinanet.backbone.body.layer4.register_forward_hook(self._get_resnet_layer4_features_hook)
    # Layers
    self.avgpool = nn.AdaptiveAvgPool2d(1)
    self.feature_map_attn = FeatureMapAttention(embed_dim=256, num_heads=attn_num_heads)
    self.combined_attn_layer = VectorAttention(embed_dim= 2048 + 256, num_heads=attn_num_heads)

    self.fc = nn.Sequential(
                  nn.Linear((2048 + 256), 256),
                  nn.ReLU(inplace=True),
                  nn.Dropout(dropout),
                  nn.Linear(256, 3)
                )
    self.FPN_linear = nn.Linear(5*256, 256)
    self.Layer2_linear = nn.Linear(512, 256)
    self.Layer3_linear = nn.Linear(1024, 256)
    self.Layer4_linear = nn.Linear(2048, 256)
    self.compress_combined_featuers_linear = nn.Linear((2048 + 256), 2048)

    self.fc2 = nn.Sequential(
                  nn.Linear((5*256), 256),
                  nn.ReLU(inplace=True),
                  nn.Dropout(dropout),
                  nn.Linear(256, 3)
                )
    self.meta_learner = nn.Linear(6,3)

    if nnn_fc_layers=="1":
      self.normal_notnormal_fc = nn.Sequential(
                                            nn.Linear(2048, 256),
                                            nn.ReLU(inplace=True),
                                            nn.Dropout(n_nn_dropout),
                                            nn.Linear(256,2)
                                          )
    elif nnn_fc_layers=="3":
      self.normal_notnormal_fc = nn.Sequential(
                                            nn.Linear(2048, 512),
                                            nn.ReLU(inplace=True),
                                            nn.Dropout(n_nn_dropout),
                                            nn.Linear(512,2)
                                          )
    if bm_fc_layers == "1":
      self.benign_malignant_fc = nn.Sequential(
                                            nn.Linear(2048, 256),
                                            nn.ReLU(inplace=True),
                                            nn.Dropout(b_m_dropout),
                                            nn.Linear(256,2)
                                          )
    elif bm_fc_layers == "2":
      self.benign_malignant_fc = nn.Sequential(
                                            nn.Linear(2048, 512),
                                            nn.ReLU(inplace=True),
                                            nn.Dropout(dropout),
                                            nn.Linear(512, 256),
                                            nn.ReLU(inplace=True),
                                            nn.Dropout(b_m_dropout),
                                            nn.Linear(256, 2)
                                          )
    elif bm_fc_layers == "3":
      self.benign_malignant_fc = nn.Sequential(
                                            nn.Linear(2048, 512),
                                            nn.ReLU(inplace=True),
                                            nn.Dropout(b_m_dropout),
                                            nn.Linear(512,2)
                                          )
    elif bm_fc_layers == "4":
      self.benign_malignant_fc = nn.Sequential(
                                            nn.Linear(2048, 1024),
                                            nn.ReLU(inplace=True),
                                            nn.Dropout(b_m_dropout),
                                            nn.Linear(1024,2)
                                          )

  def load_retinanet(self, retinanet_weight_path):
    if self.retinanet_backbone=="resnet50-gbusv":
      # No weights -> by default trainable_layers = 5 - https://github.com/pytorch/vision/blob/b1123cfd543d35d282de9bb28067e48ebec18afe/torchvision/models/detection/retinanet.py#L879
      retinanet = retinanet_resnet50_fpn_v2(num_classes=2, detections_per_img = self.detections_per_img, score_thresh=0.05,
                                            nms_thresh=0.5, min_size=self.min_size, max_size=self.max_size).to(device)
      if not self.continue_training:
        retinanet.load_state_dict(torch.load(retinanet_weight_path, map_location=device))
        print(f"Retinanet's GBUSV pre-trained weights are loaded from: {retinanet_weight_path}")
      return retinanet
    elif self.retinanet_backbone=="resnet50-coco":
      if self.rn_trainable_layers == None:
        # Defined weights -> by default trainable layers = 3 - https://github.com/pytorch/vision/blob/b1123cfd543d35d282de9bb28067e48ebec18afe/torchvision/models/detection/retinanet.py#L879
        retinanet = retinanet_resnet50_fpn_v2(weights="COCO_V1", detections_per_img=self.detections_per_img, score_thresh=0.05,
                                              nms_thresh=0.5, min_size=self.min_size, max_size=self.max_size).to(device)
      else:
        print(f"Retinanet's Trainable Backbone Layers: {self.rn_trainable_layers}")
        retinanet = retinanet_resnet50_fpn_v2(weights="COCO_V1", detections_per_img=self.detections_per_img, score_thresh=0.05,
                                              nms_thresh=0.5, min_size=self.min_size, max_size=self.max_size,
                                              trainable_backbone_layers=self.rn_trainable_layers).to(device)
      print("Retinanet's COCO pre-trained weights are loaded.")
      num_classes=2
      num_anchors = retinanet.head.classification_head.num_anchors
      in_channels = retinanet.head.classification_head.cls_logits.in_channels
      new_cls_logits = nn.Conv2d(in_channels, num_anchors * num_classes, kernel_size=3, stride=1, padding=1).to(device)
      torch.nn.init.normal_(new_cls_logits.weight, std=0.01)
      torch.nn.init.constant_(new_cls_logits.bias, -math.log((1 - 0.01) / 0.01))
      retinanet.head.classification_head.cls_logits = new_cls_logits
      retinanet.head.classification_head.num_classes = num_classes
      if self.load_pretrained_retinanet:
        checkpoint = torch.load(retinanet_weight_path, map_location=device)
        retinanet.load_state_dict(checkpoint['model_state_dict'])
        print(f"Retinanet's {checkpoint['epoch']} epoch Pretrained Weights from: {retinanet_weight_path}")
      return retinanet
    elif self.retinanet_backbone=="resnet34" or self.retinanet_backbone=="resnet18":
      # Create the backbone using ResNet34 with FPN
      backbone = resnet_fpn_backbone(backbone_name=self.retinanet_backbone, trainable_layers=3, weights="IMAGENET1K_V1").to(device)
      # Create the RetinaNet model
      retinanet = RetinaNet(backbone, num_classes=2, detections_per_img=self.detections_per_img, score_thresh=0.05, nms_thresh=0.5,
                            min_size=self.min_size, max_size=self.max_size).to(device)
      if self.regression_loss == 'giou':
        retinanet.head.regression_head._loss_type = "giou"
      return retinanet

  def load_gbcnet(self, gbcnet_weight_path):
    gbcnet = GbcNet(num_cls=3, pretrain=False, att_mode= "1")
    if not self.continue_training:
      gbcnet.load_state_dict(torch.load(gbcnet_weight_path, map_location=device))
      print(f"GBCNet's pretrained weights are loaded from: {gbcnet_weight_path}")
    gbcnet.net = gbcnet.net.float().to(device)
    return gbcnet

  def _get_retinanet_features_hook(self, module, input, output):
    self.retinanet_features = output

  def _get_gbcnet_features_hook(self, module, input, output):
    self.gbcnet_features = output

  def _get_resnet_layer1_features_hook(self, module, input, output):
    self.resnet_layer1_features = output

  def _get_resnet_layer2_features_hook(self, module, input, output):
    self.resnet_layer2_features = output

  def _get_resnet_layer3_features_hook(self, module, input, output):
    self.resnet_layer3_features = output

  def _get_resnet_layer4_features_hook(self, module, input, output):
    self.resnet_layer4_features = output

  def forward(self, inputs, targets, epoch=None):
    if self.training:
      retinanet_loss_dict = self.retinanet(inputs, targets)
      retinanet_loss = sum(loss for loss in retinanet_loss_dict.values())
      retinanet_features = self.retinanet_features
      resnet_layer1_features = self.resnet_layer1_features
      resnet_layer2_features = self.resnet_layer2_features
      resnet_layer3_features = self.resnet_layer3_features
      resnet_layer4_features = self.resnet_layer4_features

      if self.gold_curriculum==False:
        self.gold_curriculum_status_flag = print_status("Gold Curriculum Deactivated.", self.gold_curriculum_status_flag)
        if self.train_gold_bbox:
          self.train_gold_box_status_flag = print_status("Training only with gold boxes.", self.train_gold_box_status_flag)
          gbcnet_inputs, true_labels = self.prepare_gbcnet_inputs(inputs, targets, targets, epoch)
        else:
          self.train_gold_box_status_flag = print_status("Training only with predicted boxes.", self.train_gold_box_status_flag)
          pred_bboxes = self.predict_bboxes(inputs)
          gbcnet_inputs, true_labels = self.prepare_gbcnet_inputs(inputs, pred_bboxes, targets)

      else:
        self.gold_curriculum_status_flag = print_status("Gold Curriculum Activated.", self.gold_curriculum_status_flag)
        if self.current_epoch != epoch:
          self.num_gold_boxes_status_flag=False
        self.current_epoch=epoch
        if gold_box_proportion(epoch) == 1:
          self.num_gold_boxes_status_flag = print_status("All Gold Boxes are using.", self.num_gold_boxes_status_flag)
          gbcnet_inputs, true_labels = self.prepare_gbcnet_inputs(inputs, targets, targets)
        else:
          num_gold_boxes = int(gold_box_proportion(epoch)*len(targets))
          self.num_gold_boxes_status_flag = print_status(f"{num_gold_boxes} Gold Boxes, and {len(targets)-num_gold_boxes} Predicted Boxes are using.", self.num_gold_boxes_status_flag)
          pred_bboxes = self.predict_bboxes(inputs)
          selected_bboxes = targets[:num_gold_boxes]
          selected_bboxes += pred_bboxes[num_gold_boxes:]
          gbcnet_inputs, true_labels = self.prepare_gbcnet_inputs(inputs, selected_bboxes, targets)

      gbcnet_cls_logits = self.gbcnet(gbcnet_inputs)
      gbcnet_features = self.gbcnet_features

      if self.combined=='resnet-fpn':
        self.combined_status_flag = print_status("Combining features of Retinanet and GBCNet backbones.", self.combined_status_flag)
        combined_features = self.concat_feature_maps(retinanet_features, gbcnet_features)
        cls_output = self.fc(combined_features)
        classification_loss = self.criterion(cls_output.to(device), true_labels.to(device))
      elif self.combined=="resnet_layer1":
        self.combined_status_flag = print_status("Combining ResNet layer1 and GBCNet backbones.", self.combined_status_flag)
        combined_features = self.concat_feature_maps(resnet_layer1_features, gbcnet_features)
        cls_output = self.fc(combined_features)
        classification_loss = self.criterion(cls_output.to(device), true_labels.to(device))
      elif self.combined=="resnet_layer2":
        self.combined_status_flag = print_status("Combining ResNet layer2 and GBCNet backbones.", self.combined_status_flag)
        combined_features = self.concat_feature_maps(resnet_layer2_features, gbcnet_features)
        cls_output = self.fc(combined_features)
        classification_loss = self.criterion(cls_output.to(device), true_labels.to(device))
      elif self.combined=="resnet_layer3":
        self.combined_status_flag = print_status("Combining ResNet layer3 and GBCNet backbones.", self.combined_status_flag)
        combined_features = self.concat_feature_maps(resnet_layer3_features, gbcnet_features)
        cls_output = self.fc(combined_features)
        classification_loss = self.criterion(cls_output.to(device), true_labels.to(device))
      elif self.combined=="resnet_layer4":
        self.combined_status_flag = print_status("Combining ResNet layer4 and GBCNet backbones.", self.combined_status_flag)
        combined_features = self.concat_feature_maps(resnet_layer4_features, gbcnet_features)
        if not self.do_hierarchical_cls:
          cls_output = self.fc(combined_features)
          classification_loss = self.criterion(cls_output.to(device), true_labels.to(device))
        else:
          combined_features = self.compress_combined_featuers_linear(combined_features)
          classification_loss = self.hierarchical_cls(combined_features, true_labels)
      elif self.combined=="classification":
        self.combined_status_flag = print_status("Combining classifications of Retinanet and GBCNet backbones.", self.combined_status_flag)
        retinanet_features = self.concat_feature_maps(retinanet_features)
        retinanet_cls_logits = self.fc2(retinanet_features)
        combined_cls_logits = self.meta_learner(torch.cat((gbcnet_cls_logits, retinanet_cls_logits), dim=1))
        classification_loss = self.criterion(combined_cls_logits.to(device), true_labels.to(device))
      elif self.combined=='None':
        if not self.do_hierarchical_cls:
          self.combined_status_flag = print_status("Classification only using GBCNet.", self.combined_status_flag)
          classification_loss = self.criterion(gbcnet_cls_logits.to(device), true_labels.to(device))
        else:
          self.hierarchical_classification_status_flag = print_status("Hierarchical Classification is Activated.", self.hierarchical_classification_status_flag)
          gbcnet_features = self.avgpool(gbcnet_features)
          gbcnet_features = gbcnet_features.view(gbcnet_features.size(0), -1)
          classification_loss = self.hierarchical_cls(gbcnet_features, true_labels)

      total_loss = self.loss_weights["retinanet_loss_weight"] * retinanet_loss + self.loss_weights["classification_loss_weight"] * classification_loss
      return total_loss

    else:
      pred_bboxes = self.predict_bboxes(inputs)
      retinanet_features = self.retinanet_features
      resnet_layer1_features = self.resnet_layer1_features
      resnet_layer2_features = self.resnet_layer2_features
      resnet_layer3_features = self.resnet_layer3_features
      resnet_layer4_features = self.resnet_layer4_features
      gbcnet_inputs, true_labels = self.prepare_gbcnet_inputs(inputs, pred_bboxes, targets)
      gbcnet_cls_logits = self.gbcnet(gbcnet_inputs)
      gbcnet_features = self.gbcnet_features

      if self.combined=='resnet-fpn':
        combined_features = self.concat_feature_maps(retinanet_features, gbcnet_features)
        cls_output = self.fc(combined_features)
      elif self.combined=="resnet_layer1":
        combined_features = self.concat_feature_maps(resnet_layer1_features, gbcnet_features)
        cls_output = self.fc(combined_features)
      elif self.combined=="resnet_layer2":
        combined_features = self.concat_feature_maps(resnet_layer2_features, gbcnet_features)
        cls_output = self.fc(combined_features)
      elif self.combined=="resnet_layer3":
        combined_features = self.concat_feature_maps(resnet_layer3_features, gbcnet_features)
        cls_output = self.fc(combined_features)
      elif self.combined=="resnet_layer4":
        combined_features = self.concat_feature_maps(resnet_layer4_features, gbcnet_features)
        if not self.do_hierarchical_cls:
          cls_output = self.fc(combined_features)
        else:
          combined_features = self.compress_combined_featuers_linear(combined_features)
          final_h_preds = self.hierarchical_cls(combined_features, true_labels)
      elif self.combined=='classification':
        retinanet_features = self.concat_feature_maps(retinanet_features)
        retinanet_cls_logits = self.fc2(retinanet_features)
        combined_cls_logits = self.meta_learner(torch.cat((gbcnet_cls_logits, retinanet_cls_logits), dim=1))
        cls_output = combined_cls_logits
      elif self.combined=='None':
        if not self.do_hierarchical_cls:
          cls_output = gbcnet_cls_logits
        else:
          gbcnet_features = self.avgpool(gbcnet_features)
          gbcnet_features = gbcnet_features.view(gbcnet_features.size(0), -1)
          final_h_preds = self.hierarchical_cls(gbcnet_features, true_labels)

      if not self.do_hierarchical_cls:
        _, cls_pred = torch.max(cls_output, dim=1)
      else:
        cls_pred = final_h_preds

      return cls_pred, pred_bboxes

  def hierarchical_cls(self, features, true_labels):
    if self.training:
      self.hierarchical_classification_status_flag = print_status("Hierarchical Classification is Activated.", self.hierarchical_classification_status_flag)

      # Normal vs Non-Normal classification
      n_nn_output = self.normal_notnormal_fc(features)
      n_nn_true_labels = torch.tensor([0 if x == 0 else 1 for x in true_labels]) # Normal-> 0, Non-Normal -> 1

      if isinstance(self.criterion, nn.BCEWithLogitsLoss):
        n_nn_true_labels = one_hot(n_nn_true_labels, num_classes=2).float()
      n_nn_loss = self.criterion(n_nn_output.to(device), n_nn_true_labels.to(device))

      not_normal_indices = (true_labels != 0)
      if not (not_normal_indices==False).all():
        # Benign vs Malignant classification
        b_m_output = self.benign_malignant_fc(features[not_normal_indices])
        b_m_true_labels = true_labels[not_normal_indices]
        b_m_true_labels = torch.tensor([0 if x==1 else 1 for x in b_m_true_labels]) # Benign -> 0, Malignant -> 1

        if isinstance(self.criterion, nn.BCEWithLogitsLoss):
          b_m_true_labels = one_hot(b_m_true_labels, num_classes=2).float()
        b_m_loss = self.criterion(b_m_output.to(device), b_m_true_labels.to(device))
      else:
        b_m_loss = 0
      loss = self.loss_weights["n_nn_loss_weight"] * n_nn_loss + self.loss_weights["b_m_loss_weight"] * b_m_loss
      return loss
    else:
      n_nn_output = self.normal_notnormal_fc(features)
      _, n_nn_cls_pred = torch.max(n_nn_output, dim=1)
      final_h_preds = torch.zeros_like(n_nn_cls_pred)
      not_normal_indices = (n_nn_cls_pred != 0)
      if not (not_normal_indices==False).all():
        b_m_output = self.benign_malignant_fc(features[not_normal_indices])
        _, b_m_cls_pred = torch.max(b_m_output, dim=1)
        b_m_cls_pred += 1
        final_h_preds[not_normal_indices] = b_m_cls_pred
      return final_h_preds


  def predict_bboxes(self, inputs):
    self.retinanet.eval()
    with torch.inference_mode():
      predictions = self.retinanet(inputs)
      foreground_preds = filter_background(predictions)
    if self.training:
      self.retinanet.train()
    return foreground_preds

  def va_blur(self, img, epoch):
    self.gbcnet_va_curriculum_flag = print_status("GBCNet VA Curriculum Activated.", self.gbcnet_va_curriculum_flag)
    if epoch <= 10:
      gaussian_blur = GaussianBlur((17,17), 8)
      img = gaussian_blur(img)
    elif epoch > 10 and epoch <= 15:
      gaussian_blur = GaussianBlur((9,9), 4)
      img = gaussian_blur(img)
    elif epoch > 15 and epoch <= 20:
      gaussian_blur = GaussianBlur((5,5), 2)
      img = gaussian_blur(img)
    elif epoch > 20 and epoch <= 25:
      gaussian_blur = GaussianBlur((3,3), 1)
      img = gaussian_blur(img)
    elif epoch > 25 and epoch <= 30:
      gaussian_blur = GaussianBlur((1,1), 0.5)
      img = gaussian_blur(img)
    else:
      return img
    return img

  def prepare_gbcnet_inputs(self, inputs, predictions, targets, epoch=None):
    true_labels = []
    gbcnet_inputs = []
    for i in range(len(inputs)):
      img = inputs[i]
      if (self.gbcnet_va_curriculum) and (epoch is not None):
        img = self.va_blur(img, epoch)
      true_labels.append(targets[i]['true_labels'])
      pred_bboxes = predictions[i]['boxes']
      #visualize(img)
      if len(pred_bboxes) > 0:
        for pred_bbox in pred_bboxes:
          pred_bbox_area = (int(pred_bbox[2])-int(pred_bbox[0])) * (int(pred_bbox[3])-int(pred_bbox[1]))
          if pred_bbox_area > 0:
            img = crop_image(img, pred_bbox, self.p)
            img = F.interpolate(img.unsqueeze(0), size=(self.height, self.width), mode='bilinear', align_corners=False).squeeze(0)
      gbcnet_inputs.append(img)
    return torch.stack(gbcnet_inputs), torch.stack(true_labels)

  def concat_feature_maps(self, rn_fms, gb_fm=None):
    if self.combined=='resnet-fpn':
      rn_fms = list(rn_fms.values())
      if not self.no_attn:
        self.no_attn_status_flag = print_status("Self attention on Retinanet's feature maps are applied.", self.no_attn_status_flag)
        rn_fms = [self.feature_map_attn(x) for x in rn_fms]
      rn_fms = [self.avgpool(x) for x in rn_fms]
      rn_fms = [x.view(x.size(0), -1) for x in rn_fms]
      rn_rms = torch.stack(rn_fms)
      rn_fms = torch.cat(rn_fms, dim=-1)
      rn_fms = self.FPN_linear(rn_fms)
      gb_fm = self.avgpool(gb_fm)
      gb_fm = gb_fm.view(gb_fm.size(0), -1)
      concatenated_features = torch.cat([rn_fms, gb_fm], dim = 1)
    elif self.combined in ["resnet_layer1", "resnet_layer2", "resnet_layer3", "resnet_layer4"]:
      rn_fms = self.avgpool(rn_fms)
      rn_fms = rn_fms.reshape(rn_fms.size(0), -1)
      if self.combined=="resnet_layer2":
        rn_fms = self.Layer2_linear(rn_fms)
      elif self.combined=="resnet_layer3":
        rn_fms = self.Layer3_linear(rn_fms)
      elif self.combined=="resnet_layer4":
        rn_fms = self.Layer4_linear(rn_fms)
      gb_fm = self.avgpool(gb_fm)
      gb_fm = gb_fm.reshape(gb_fm.size(0),-1)
      concatenated_features = torch.cat([rn_fms, gb_fm], dim=1)
      if not no_attn:
        self.no_attn_status_flag = print_status("Self attention is applied on combined features.", self.no_attn_status_flag)
        concatenated_features = self.combined_attn_layer(concatenated_features)
    elif self.combined=='classification':
      rn_fms = list(rn_fms.values())
      if not self.no_attn:
        self.no_attn_status_flag = print_status("Self attention on Retinanet's feature maps are applied.", self.no_attn_status_flag)
        rn_fms = [self.feature_map_attn(x) for x in rn_fms]
      rn_fms = [self.avgpool(x) for x in rn_fms]
      rn_fms = [x.view(x.size(0), -1) for x in rn_fms]
      concatenated_features = torch.cat(rn_fms, dim = 1)
    return concatenated_features


## Training

In [ ]:
def train_model(gbcret, num_epochs, optimizer, va_curriculum, batch_size, save_model_dir, train_transform, eval_transform,
                freeze_after_20=False, best_acc=0, num_worse_epochs=0, patience=0, checkpoint_save_rate=5,
                last_epoch=0, train_scores={'acc':[],'mAP':[]}, val_scores={'acc':[],'mAP':[]}, lr_scheduler=None):

  # Validation DataLoader
  val_dataset = InputDataset(imgs_folder_path, val_txt_path, gold_file_path, to_blur=False, img_transforms=eval_transform)
  val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn= collate_fn)

  best_epoch = 0
  best_model = None
  show_num_images = True

  os.makedirs(f'{save_model_dir}/checkpoints', exist_ok=True)

  print(f"Last Epoch: {last_epoch}, Start Epoch: {last_epoch+1}, End Epoch: {num_epochs}")
  print(f"VA Curriculum: {va_curriculum}")
  print(f"Train Scores Length: {len(train_scores['acc'])}, Val Scores Length: {len(val_scores['acc'])}")
  print(f"Best accuracy so far: {best_acc}")
  print(f"Patience so far: {num_worse_epochs}")
  for epoch in range(last_epoch+1, num_epochs+1):

    print(f'\nEpoch {epoch}:')
    start_time = time.time()

    if va_curriculum:
      if epoch <= 10:
        train_dataset = InputDataset(imgs_folder_path, train_txt_path, gold_file_path, to_blur=True, blur_kernel_size=(65,65), sigma=16, img_transforms=train_transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn= collate_fn)
      elif epoch >10 and epoch <=15:
        train_dataset = InputDataset(imgs_folder_path, train_txt_path, gold_file_path, to_blur=True, blur_kernel_size=(33,33), sigma=8, img_transforms=train_transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn= collate_fn)
      elif epoch >15 and epoch <=20:
        train_dataset = InputDataset(imgs_folder_path, train_txt_path, gold_file_path, to_blur=True, blur_kernel_size=(17,17), sigma=4, img_transforms=train_transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn= collate_fn)
      elif epoch >20 and epoch <=25:
        train_dataset = InputDataset(imgs_folder_path, train_txt_path, gold_file_path, to_blur=True, blur_kernel_size=(9,9), sigma=2, img_transforms=train_transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn= collate_fn)
      elif epoch >25 and epoch <=30:
        train_dataset = InputDataset(imgs_folder_path, train_txt_path, gold_file_path, to_blur=True, blur_kernel_size=(5,5), sigma=1, img_transforms=train_transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn= collate_fn)
      else:
        train_dataset = InputDataset(imgs_folder_path, train_txt_path, gold_file_path, to_blur=False, img_transforms= train_transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn= collate_fn)
    else:
      train_dataset = InputDataset(imgs_folder_path, train_txt_path, gold_file_path, to_blur=False, img_transforms=train_transform)
      train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn= collate_fn)

    if show_num_images == True:
      print(f"Number of Train images: {len(train_dataset)}")
      print(f"Number of Test images: {len(val_dataset)}")
      show_num_images = False

    # Training Loop
    gbcret.train()
    running_loss = 0.0
    total_step = len(train_loader)
    for i, data in enumerate(train_loader, 0):
      images_ids, inputs, targets = data[0],\
                                    [image.to(device) for image in data[1]],\
                                    [{k: v.to(device) if torch.is_tensor(v) else v for k, v in t.items()} for t in data[2]]

      optimizer.zero_grad()

      loss = gbcret(inputs, targets, epoch)

      loss.backward()

      optimizer.step()

      running_loss += loss.item()

    train_loss = running_loss/total_step
    print(f"Train Loss: {train_loss}, Total step: {total_step}")


    # Evaluating Model
    gbcret.eval()
    print("Train Evaluation:")
    train_acc, train_mAP, train_cfm, train_y_true, train_y_pred, train_fnames, _ = evaluate_model(gbcret, train_loader)
    train_scores['acc'].append(train_acc)
    train_scores['mAP'].append(train_mAP)

    print("Val Evaluation:")
    val_acc, val_mAP, val_cfm, val_y_true, val_y_pred, val_fnames, _  = evaluate_model(gbcret, val_loader)
    val_scores['acc'].append(val_acc)
    val_scores['mAP'].append(val_mAP)

    if (lr_scheduler is not None) and (epoch>=50):
      lr_scheduler.step(val_acc)

    # Early Stop
    if val_acc > best_acc:
      best_acc = val_acc
      best_epoch = epoch
      best_model = copy.deepcopy(gbcret)
      num_worse_epochs = 0
    else:
      if patience > 0:
        num_worse_epochs += 1

    if ((epoch) % checkpoint_save_rate == 0) or (num_worse_epochs > patience):
      random_state = random.getstate()
      np_random_state = np.random.get_state()
      torch_random_state = torch.get_rng_state()
      torch.save({'model_state_dict': gbcret.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'best_acc': best_acc,
                  'train_scores': train_scores,
                  'val_scores': val_scores,
                  'epoch': epoch,
                  'num_worse_epochs': num_worse_epochs,
                  'random_state': random_state,
                  'np_random_state': np_random_state,
                  'torch_random_state': torch_random_state},
                  f'{save_model_dir}/checkpoints/checkpoint_epoch_{epoch}.pth')
      print(f"\nCheckpoint is saved to: {save_model_dir}/checkpoints/checkpoint_epoch_{epoch}.pth\n")

      if best_model is not None:
        torch.save({'model_state_dict': best_model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'best_acc': best_acc,
                    'epoch': best_epoch},
                    f'{save_model_dir}/checkpoints/best_model_epoch_{best_epoch}.pth')
        print(f'Best model is saved to: {save_model_dir}/checkpoints/best_model_epoch_{best_epoch}.pth')
        print(f"Best model was for epoch {best_epoch} with best Acc: {best_acc}")

      else:
        print("\nNo Best model to save!")

      if num_worse_epochs > patience:
        print(f"\nNo better results for last {patience} epochs!\nBest model was for epoch {best_epoch} with best mAP: {best_acc}")
        break


    # Time
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Epoch time: {elapsed_time}")
    remaining_epochs = num_epochs - epoch
    remaining_time = round(elapsed_time * remaining_epochs)
    hours, remainder = divmod(remaining_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f"Estimated time remaining: {hours}:{minutes}:{seconds}")

    print(f"GPU Memory Used: {torch.cuda.memory_allocated()}")

## Experiments

### GBCRet, No Combination, with gbcnet VA curriculum

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="None"
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn,
                         gbcnet_va_curriculum=gbcnet_va_curriculum).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=5, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:05<00:00, 25.7MB/s]


Retinanet's COCO pre-trained weights are loaded.
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 200
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0

Epoch 1:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Classification only using GBCNet.
Train Loss: 2.4473005817050026, Total step: 21
Train Evaluation:
mIoU = 0.0013646513689309359, median IoU: 0.0, mAP = 0.0, mAP_50 = 0.0, mAP_75 = 0.0
TP: 3, FN: 29, FP: 1101
Precision: 0.002717391304347826
Recall: 0.09375
Acc: 0.4545 Spec: 0.8890 Sens: 0.2691
Val Evaluation:
mIoU = 0.001426889095455408, median IoU: 0.0, mAP = 0.0, mAP_50 = 0.0, mAP_75 = 0.0
TP: 0, FN: 4, FP: 118
Precision: 0.0
Recall: 0.0
Acc: 0.3115 Spec: 0.7750 Sens: 0.0714
Epoch time: 202.37799835205078
Estimated time remaining: 11

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="None"
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout = dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA/checkpoints/checkpoint_epoch_15.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 55.6MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 15, Start Epoch: 16, End Epoch: 200
VA Curriculum: False
Train Scores Length: 15, Val Scores Length: 15
Best accuracy so far: 0.5573770491803278

Epoch 16:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Classification only using GBCNet.
Train Loss: 0.7952530469213214, Total step: 21
Train Evaluation:
mIoU = 0.6843181848526001, median IoU: 0.7321344614028931, mAP = 0.382, mAP_50 = 0.812, mAP_75 = 0.3
TP: 1087, FN: 0, FP: 46
Precision: 0.9593998234774934
Recall: 1.0
Acc: 0.6929 Spec: 0.8187 Sens: 0.8700
Val Evaluation:
mIoU = 0.5671966671943665, median IoU: 0.6148629784584045, mAP = 0.217, mAP_50 = 0.551, mAP_75 = 0.115
TP: 106, FN: 0, FP: 16
Precision: 0.8688524590163934
Recall: 1.0
Acc: 0.6885 Spec: 0.7500 Sens: 0.7857
Epoch time: 176.3911907672882
Estimated time remaining: 9:0:56
GPU Memory Used: 1567335424

Epoch 17:
Train L

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="None"
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout = dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA/checkpoints/checkpoint_epoch_45.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 85.7MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 45, Start Epoch: 46, End Epoch: 200
VA Curriculum: False
Train Scores Length: 45, Val Scores Length: 45
Best accuracy so far: 0.8770491803278688

Epoch 46:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Classification only using GBCNet.
Train Loss: 0.6199247922216143, Total step: 21
Train Evaluation:
mIoU = 0.7472525835037231, median IoU: 0.7796764969825745, mAP = 0.483, mAP_50 = 0.911, mAP_75 = 0.469
TP: 1118, FN: 0, FP: 15
Precision: 0.9867608120035305
Recall: 1.0
Acc: 0.8438 Spec: 0.9769 Sens: 0.7489
Val Evaluation:
mIoU = 0.6230027079582214, median IoU: 0.6579837203025818, mAP = 0.265, mAP_50 = 0.697, mAP_75 = 0.163
TP: 115, FN: 0, FP: 7
Precision: 0.9426229508196722
Recall: 1.0
Acc: 0.8689 Spec: 0.9375 Sens: 0.7857
Epoch time: 211.1636083126068
Estimated time remaining: 9:1:59
GPU Memory Used: 1202911744

Epoch 47:
Train 

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="None"
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout = dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA/checkpoints/checkpoint_epoch_85.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 64.4MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 85, Start Epoch: 86, End Epoch: 200
VA Curriculum: False
Train Scores Length: 85, Val Scores Length: 85
Best accuracy so far: 0.8852459016393442

Epoch 86:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Classification only using GBCNet.
Train Loss: 0.5180145360174633, Total step: 21
Train Evaluation:
mIoU = 0.7843860387802124, median IoU: 0.8052176237106323, mAP = 0.553, mAP_50 = 0.967, mAP_75 = 0.562
TP: 1124, FN: 0, FP: 9
Precision: 0.9920564872021183
Recall: 1.0
Acc: 0.8844 Spec: 0.9868 Sens: 0.8251
Val Evaluation:
mIoU = 0.6478790640830994, median IoU: 0.6917141675949097, mAP = 0.295, mAP_50 = 0.809, mAP_75 = 0.169
TP: 117, FN: 0, FP: 5
Precision: 0.9590163934426229
Recall: 1.0
Acc: 0.8443 Spec: 0.9500 Sens: 0.7143
Epoch time: 265.8120217323303
Estimated time remaining: 8:25:3
GPU Memory Used: 1202911744

Epoch 87:
Train L

### GBCRet, No Combination, with gbcnet VA curriculum, start from retinanet 50 epoch pretrained weights

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="None"
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=5, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 73.5MB/s]


Retinanet's COCO pre-trained weights are loaded.
Retinanet's 50 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 200
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0

Epoch 1:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Classification only using GBCNet.
Train Loss: 1.2911417512666612, Total step: 21
Train Evaluation:
mIoU = 0.7506575584411621, median IoU: 0.7779878973960876, mAP = 0.496, mAP_50 = 0.92, mAP_75 = 0.469
TP: 1117, FN: 0, FP: 16
Precision: 0.9858781994704324
Recall: 1.0
Acc: 0.4731 Spec: 0.6165 Sens: 0.6951
Val Evaluation:
mIoU = 0.633614182472229, median IoU: 0.6749396920204163, mAP = 0.27

**missing one step of training due do notebook's saving problem**

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="None"
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA, 50 epoch Pretrained RN"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout = dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA, 50 epoch Pretrained RN/checkpoints/checkpoint_epoch_45.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:03<00:00, 46.6MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 45, Start Epoch: 46, End Epoch: 200
VA Curriculum: False
Train Scores Length: 45, Val Scores Length: 45
Best accuracy so far: 0.8934426229508197

Epoch 46:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Classification only using GBCNet.
Train Loss: 0.5422426490556627, Total step: 21
Train Evaluation:
mIoU = 0.7880745530128479, median IoU: 0.8101170063018799, mAP = 0.563, mAP_50 = 0.978, mAP_75 = 0.594
TP: 1127, FN: 0, FP: 6
Precision: 0.9947043248014121
Recall: 1.0
Acc: 0.8605 Spec: 0.9791 Sens: 0.7937
Val Evaluation:
mIoU = 0.6563865542411804, median IoU: 0.6972051858901978, mAP = 0.307, mAP_50 = 0.812, mAP_75 = 0.174
TP: 117, FN: 0, FP: 5
Precision: 0.9590163934426229
Recall: 1.0
Acc: 0.8770 Spec: 0.9750 Sens: 0.7381
Epoch time: 200.1976034641266
Estimated time remaining: 8:33:50
GPU Memory Used: 1202911744

Epoch 47:
Train 

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="None"
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA, 50 epoch Pretrained RN"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout = dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA, 50 epoch Pretrained RN/checkpoints/checkpoint_epoch_100.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 75.5MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 100, Start Epoch: 101, End Epoch: 200
VA Curriculum: False
Train Scores Length: 100, Val Scores Length: 100
Best accuracy so far: 0.8934426229508197

Epoch 101:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Classification only using GBCNet.
Train Loss: 0.4357964510009402, Total step: 21
Train Evaluation:
mIoU = 0.8173514604568481, median IoU: 0.8307783603668213, mAP = 0.622, mAP_50 = 0.99, mAP_75 = 0.729
TP: 1129, FN: 0, FP: 4
Precision: 0.9964695498676082
Recall: 1.0
Acc: 0.9073 Spec: 0.9846 Sens: 0.8475
Val Evaluation:
mIoU = 0.65495365858078, median IoU: 0.6895843744277954, mAP = 0.303, mAP_50 = 0.779, mAP_75 = 0.146
TP: 117, FN: 0, FP: 5
Precision: 0.9590163934426229
Recall: 1.0
Acc: 0.8607 Spec: 0.9625 Sens: 0.7381
Epoch time: 163.53824186325073
Estimated time remaining: 4:29:50
GPU Memory Used: 1202911744

Epoch 102:
Tr

### GBCRet, No Combination, with gbcnet VA curriculum, start from retinanet 30 epoch pretrained weights

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="None"
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_30.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA, 30 epoch Pretrained RN"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=5, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:10<00:00, 14.6MB/s]


Retinanet's COCO pre-trained weights are loaded.
Retinanet's 30 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_30.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 100
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0

Epoch 1:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Classification only using GBCNet.
Train Loss: 1.3478054404258728, Total step: 21
Train Evaluation:
mIoU = 0.7207642197608948, median IoU: 0.7582679390907288, mAP = 0.442, mAP_50 = 0.872, mAP_75 = 0.387
TP: 1109, FN: 0, FP: 24
Precision: 0.9788172992056487
Recall: 1.0
Acc: 0.4687 Spec: 0.6165 Sens: 0.7175
Val Evaluation:
mIoU = 0.6086874604225159, median IoU: 0.6613789200782776, mAP = 0.

**missing one step of training due do notebook's saving problem**

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="None"
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA, 30 epoch Pretrained RN"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout = dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA, 30 epoch Pretrained RN/checkpoints/checkpoint_epoch_40.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:03<00:00, 39.8MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 40, Start Epoch: 41, End Epoch: 100
VA Curriculum: False
Train Scores Length: 40, Val Scores Length: 40
Best accuracy so far: 0.8770491803278688

Epoch 41:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Classification only using GBCNet.
Train Loss: 0.6166344994590396, Total step: 21
Train Evaluation:
mIoU = 0.7726113796234131, median IoU: 0.795008659362793, mAP = 0.533, mAP_50 = 0.956, mAP_75 = 0.533
TP: 1125, FN: 0, FP: 8
Precision: 0.9929390997352162
Recall: 1.0
Acc: 0.8605 Spec: 0.9879 Sens: 0.7489
Val Evaluation:
mIoU = 0.6371034383773804, median IoU: 0.6716150045394897, mAP = 0.283, mAP_50 = 0.768, mAP_75 = 0.152
TP: 116, FN: 0, FP: 6
Precision: 0.9508196721311475
Recall: 1.0
Acc: 0.8852 Spec: 0.9750 Sens: 0.7619
Epoch time: 245.779443025589
Estimated time remaining: 4:1:41
GPU Memory Used: 1561410048

Epoch 42:
Train Los

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="None"
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA, 30 epoch Pretrained RN"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout = dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA, 30 epoch Pretrained RN/checkpoints/checkpoint_epoch_85.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 67.5MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 85, Start Epoch: 86, End Epoch: 100
VA Curriculum: False
Train Scores Length: 85, Val Scores Length: 85
Best accuracy so far: 0.9016393442622951

Epoch 86:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Classification only using GBCNet.
Train Loss: 0.48593905851954505, Total step: 21
Train Evaluation:
mIoU = 0.8012787699699402, median IoU: 0.8167924284934998, mAP = 0.587, mAP_50 = 0.979, mAP_75 = 0.637
TP: 1128, FN: 0, FP: 5
Precision: 0.9955869373345102
Recall: 1.0
Acc: 0.8853 Spec: 0.9857 Sens: 0.8296
Val Evaluation:
mIoU = 0.6536242365837097, median IoU: 0.690125048160553, mAP = 0.311, mAP_50 = 0.812, mAP_75 = 0.148
TP: 117, FN: 0, FP: 5
Precision: 0.9590163934426229
Recall: 1.0
Acc: 0.8607 Spec: 0.9750 Sens: 0.7143
Epoch time: 163.05443477630615
Estimated time remaining: 0:38:3
GPU Memory Used: 1202911744

Epoch 87:
Train 

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="None"
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA, 30 epoch Pretrained RN"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout = dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA, 30 epoch Pretrained RN/checkpoints/checkpoint_epoch_100.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 141MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 100, Start Epoch: 101, End Epoch: 200
VA Curriculum: False
Train Scores Length: 100, Val Scores Length: 100
Best accuracy so far: 0.9016393442622951

Epoch 101:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Classification only using GBCNet.
Train Loss: 0.45058775657699224, Total step: 21
Train Evaluation:
mIoU = 0.8089567422866821, median IoU: 0.8223928809165955, mAP = 0.604, mAP_50 = 0.99, mAP_75 = 0.696
TP: 1128, FN: 0, FP: 5
Precision: 0.9955869373345102
Recall: 1.0
Acc: 0.8959 Spec: 0.9824 Sens: 0.8206
Val Evaluation:
mIoU = 0.6558488607406616, median IoU: 0.6928252577781677, mAP = 0.306, mAP_50 = 0.812, mAP_75 = 0.155
TP: 117, FN: 0, FP: 5
Precision: 0.9590163934426229
Recall: 1.0
Acc: 0.8607 Spec: 0.9750 Sens: 0.7143
Epoch time: 171.3323724269867
Estimated time remaining: 4:42:42
GPU Memory Used: 1202911744

Epoch 102:


KeyboardInterrupt: 

### GBCRet, No Combination, with gbcnet VA curriculum, start from retinanet 100 epoch pretrained weights

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="None"
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_100.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA, 100 epoch Pretrained RN"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=5, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 69.1MB/s]


Retinanet's COCO pre-trained weights are loaded.
Retinanet's 100 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_100.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 100
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0

Epoch 1:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Classification only using GBCNet.
Train Loss: 1.2181064571653093, Total step: 21
Train Evaluation:
mIoU = 0.7943846583366394, median IoU: 0.8094071745872498, mAP = 0.576, mAP_50 = 0.979, mAP_75 = 0.633
TP: 1127, FN: 0, FP: 6
Precision: 0.9947043248014121
Recall: 1.0
Acc: 0.4828 Spec: 0.6209 Sens: 0.7444
Val Evaluation:
mIoU = 0.6619246602058411, median IoU: 0.7005577683448792, mAP = 0

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="None"
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA, 100 epoch Pretrained RN"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout = dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA, 100 epoch Pretrained RN/checkpoints/checkpoint_epoch_35.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:03<00:00, 50.0MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 35, Start Epoch: 36, End Epoch: 100
VA Curriculum: False
Train Scores Length: 35, Val Scores Length: 35
Best accuracy so far: 0.8278688524590164

Epoch 36:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Classification only using GBCNet.
Train Loss: 0.5303656927176884, Total step: 21
Train Evaluation:
mIoU = 0.8115599751472473, median IoU: 0.822394847869873, mAP = 0.61, mAP_50 = 0.99, mAP_75 = 0.711
TP: 1130, FN: 0, FP: 3
Precision: 0.9973521624007061
Recall: 1.0
Acc: 0.8676 Spec: 0.9802 Sens: 0.8027
Val Evaluation:
mIoU = 0.6548830270767212, median IoU: 0.70000821352005, mAP = 0.305, mAP_50 = 0.801, mAP_75 = 0.154
TP: 118, FN: 0, FP: 4
Precision: 0.9672131147540983
Recall: 1.0
Acc: 0.8279 Spec: 0.9625 Sens: 0.6429
Epoch time: 162.54987502098083
Estimated time remaining: 2:53:23
GPU Memory Used: 1202911744

Epoch 37:
Train Loss

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="None"
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA, 100 epoch Pretrained RN"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout = dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA, 100 epoch Pretrained RN/checkpoints/checkpoint_epoch_80.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 60.1MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 80, Start Epoch: 81, End Epoch: 100
VA Curriculum: False
Train Scores Length: 80, Val Scores Length: 80
Best accuracy so far: 0.8770491803278688

Epoch 81:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Classification only using GBCNet.
Train Loss: 0.43603430617423283, Total step: 21
Train Evaluation:
mIoU = 0.8290185332298279, median IoU: 0.8421105742454529, mAP = 0.648, mAP_50 = 0.99, mAP_75 = 0.798
TP: 1128, FN: 0, FP: 5
Precision: 0.9955869373345102
Recall: 1.0
Acc: 0.8870 Spec: 0.9835 Sens: 0.8565
Val Evaluation:
mIoU = 0.6589586734771729, median IoU: 0.6920398473739624, mAP = 0.306, mAP_50 = 0.781, mAP_75 = 0.163
TP: 118, FN: 0, FP: 4
Precision: 0.9672131147540983
Recall: 1.0
Acc: 0.8525 Spec: 0.9625 Sens: 0.7381
Epoch time: 163.50698900222778
Estimated time remaining: 0:51:47
GPU Memory Used: 1202911744

Epoch 82:
Train

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="None"
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA, 100 epoch Pretrained RN"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout = dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA, 100 epoch Pretrained RN/checkpoints/checkpoint_epoch_100.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 80.2MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 100, Start Epoch: 101, End Epoch: 200
VA Curriculum: False
Train Scores Length: 100, Val Scores Length: 100
Best accuracy so far: 0.8852459016393442

Epoch 101:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Classification only using GBCNet.
Train Loss: 0.4062755292370206, Total step: 21
Train Evaluation:
mIoU = 0.8339440822601318, median IoU: 0.8439026474952698, mAP = 0.659, mAP_50 = 0.99, mAP_75 = 0.818
TP: 1130, FN: 0, FP: 3
Precision: 0.9973521624007061
Recall: 1.0
Acc: 0.9082 Spec: 0.9824 Sens: 0.8610
Val Evaluation:
mIoU = 0.6565818190574646, median IoU: 0.6834329962730408, mAP = 0.303, mAP_50 = 0.785, mAP_75 = 0.16
TP: 118, FN: 0, FP: 4
Precision: 0.9672131147540983
Recall: 1.0
Acc: 0.8607 Spec: 0.9500 Sens: 0.7619
Epoch time: 164.16676688194275
Estimated time remaining: 4:30:53
GPU Memory Used: 1202911744

Epoch 102:
T

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="None"
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA, 100 epoch Pretrained RN"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout = dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, no combination, gbcnet VA, 100 epoch Pretrained RN/checkpoints/checkpoint_epoch_130.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 57.8MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 130, Start Epoch: 131, End Epoch: 200
VA Curriculum: False
Train Scores Length: 130, Val Scores Length: 130
Best accuracy so far: 0.9016393442622951

Epoch 131:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Classification only using GBCNet.
Train Loss: 0.3613613872301011, Total step: 21
Train Evaluation:
mIoU = 0.8440844416618347, median IoU: 0.854506254196167, mAP = 0.682, mAP_50 = 0.99, mAP_75 = 0.873
TP: 1131, FN: 0, FP: 2
Precision: 0.9982347749338041
Recall: 1.0
Acc: 0.9117 Spec: 0.9857 Sens: 0.8655
Val Evaluation:
mIoU = 0.6575266718864441, median IoU: 0.6851903796195984, mAP = 0.308, mAP_50 = 0.814, mAP_75 = 0.163
TP: 118, FN: 0, FP: 4
Precision: 0.9672131147540983
Recall: 1.0
Acc: 0.8852 Spec: 0.9625 Sens: 0.7857
Epoch time: 158.65400862693787
Estimated time remaining: 3:2:27
GPU Memory Used: 1202911744

Epoch 132:
Tr

KeyboardInterrupt: 

### GBCRet, Combination layer1, gbcnet VA, 50 epoch pretrained RN

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer1"
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Combination layer1, gbcnet VA, 50 epoch pretrained RN"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=5, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 54.9MB/s]


Retinanet's COCO pre-trained weights are loaded.
Retinanet's 50 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 100
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0

Epoch 1:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer1 and GBCNet backbones.
Train Loss: 1.40419830594744, Total step: 21
Train Evaluation:
mIoU = 0.7506569027900696, median IoU: 0.7779107093811035, mAP = 0.496, mAP_50 = 0.92, mAP_75 = 0.469
TP: 1117, FN: 0, FP: 16
Precision: 0.9858781994704324
Recall: 1.0
Acc: 0.4492 Spec: 1.0000 Sens: 0.0000
Val Evaluation:
mIoU = 0.6336131691932678, median IoU: 0.6749560832977295,

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer1"
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Combination layer1, gbcnet VA, 50 epoch pretrained RN"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout = dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Combination layer1, gbcnet VA, 50 epoch pretrained RN/checkpoints/checkpoint_epoch_75.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:11<00:00, 13.9MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 75, Start Epoch: 76, End Epoch: 100
VA Curriculum: False
Train Scores Length: 75, Val Scores Length: 75
Best accuracy so far: 0.860655737704918

Epoch 76:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer1 and GBCNet backbones.
Train Loss: 0.5283152418477195, Total step: 21
Train Evaluation:
mIoU = 0.8075834512710571, median IoU: 0.8217640519142151, mAP = 0.603, mAP_50 = 0.989, mAP_75 = 0.682
TP: 1129, FN: 0, FP: 4
Precision: 0.9964695498676082
Recall: 1.0
Acc: 0.8764 Spec: 0.9912 Sens: 0.7668
Val Evaluation:
mIoU = 0.6525922417640686, median IoU: 0.6948491334915161, mAP = 0.305, mAP_50 = 0.792, mAP_75 = 0.171
TP: 117, FN: 0, FP: 5
Precision: 0.9590163934426229
Recall: 1.0
Acc: 0.7787 Spec: 0.9875 Sens: 0.5000
Epoch time: 154.9557888507843
Estimated time remaining: 1:1:59
GPU Memory Used: 1202911744

Epoch 

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer1"
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Combination layer1, gbcnet VA, 50 epoch pretrained RN"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout = dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Combination layer1, gbcnet VA, 50 epoch pretrained RN/checkpoints/checkpoint_epoch_100.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 71.9MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 100, Start Epoch: 101, End Epoch: 200
VA Curriculum: False
Train Scores Length: 100, Val Scores Length: 100
Best accuracy so far: 0.8770491803278688

Epoch 101:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer1 and GBCNet backbones.
Train Loss: 0.4367680691537403, Total step: 21
Train Evaluation:
mIoU = 0.8170576691627502, median IoU: 0.8313516974449158, mAP = 0.622, mAP_50 = 0.99, mAP_75 = 0.729
TP: 1129, FN: 0, FP: 4
Precision: 0.9964695498676082
Recall: 1.0
Acc: 0.9020 Spec: 0.9857 Sens: 0.8206
Val Evaluation:
mIoU = 0.654608964920044, median IoU: 0.6921132802963257, mAP = 0.303, mAP_50 = 0.779, mAP_75 = 0.148
TP: 117, FN: 0, FP: 5
Precision: 0.9590163934426229
Recall: 1.0
Acc: 0.8115 Spec: 0.9625 Sens: 0.6190
Epoch time: 163.96997833251953
Estimated time remaining: 4:30:33
GPU Memory Used: 1202911744



In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer1"
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Combination layer1, gbcnet VA, 50 epoch pretrained RN"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout = dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Combination layer1, gbcnet VA, 50 epoch pretrained RN/checkpoints/checkpoint_epoch_135.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:05<00:00, 26.0MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 135, Start Epoch: 136, End Epoch: 200
VA Curriculum: False
Train Scores Length: 135, Val Scores Length: 135
Best accuracy so far: 0.8852459016393442

Epoch 136:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer1 and GBCNet backbones.
Train Loss: 0.3717308058625176, Total step: 21
Train Evaluation:
mIoU = 0.8322151899337769, median IoU: 0.8446263670921326, mAP = 0.656, mAP_50 = 0.99, mAP_75 = 0.81
TP: 1129, FN: 0, FP: 4
Precision: 0.9964695498676082
Recall: 1.0
Acc: 0.9109 Spec: 0.9879 Sens: 0.8655
Val Evaluation:
mIoU = 0.658858597278595, median IoU: 0.6883084774017334, mAP = 0.308, mAP_50 = 0.79, mAP_75 = 0.158
TP: 118, FN: 0, FP: 4
Precision: 0.9672131147540983
Recall: 1.0
Acc: 0.8607 Spec: 0.9625 Sens: 0.7381
Epoch time: 178.80266404151917
Estimated time remaining: 3:10:43
GPU Memory Used: 1202911744

Ep

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer1"
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Combination layer1, gbcnet VA, 50 epoch pretrained RN"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout = dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Combination layer1, gbcnet VA, 50 epoch pretrained RN/checkpoints/checkpoint_epoch_165.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 81.4MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 165, Start Epoch: 166, End Epoch: 200
VA Curriculum: False
Train Scores Length: 165, Val Scores Length: 165
Best accuracy so far: 0.8934426229508197

Epoch 166:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer1 and GBCNet backbones.
Train Loss: 0.3285434175105322, Total step: 21
Train Evaluation:
mIoU = 0.8389790058135986, median IoU: 0.8504884243011475, mAP = 0.672, mAP_50 = 0.99, mAP_75 = 0.843
TP: 1130, FN: 0, FP: 3
Precision: 0.9973521624007061
Recall: 1.0
Acc: 0.9373 Spec: 0.9857 Sens: 0.9193
Val Evaluation:
mIoU = 0.6585366725921631, median IoU: 0.6866092681884766, mAP = 0.311, mAP_50 = 0.802, mAP_75 = 0.174
TP: 117, FN: 0, FP: 5
Precision: 0.9590163934426229
Recall: 1.0
Acc: 0.8689 Spec: 0.9500 Sens: 0.7619
Epoch time: 175.36175966262817
Estimated time remaining: 1:39:22
GPU Memory Used: 1202911744


### GBCRet, Combination layer2, gbcnet VA, 50 epoch pretrained RN

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer2"
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Combination layer2, gbcnet VA, 50 epoch pretrained RN"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=5, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:03<00:00, 41.1MB/s]


Retinanet's COCO pre-trained weights are loaded.
Retinanet's 50 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 100
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0

Epoch 1:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer2 and GBCNet backbones.
Train Loss: 1.424300403822036, Total step: 21
Train Evaluation:
mIoU = 0.750663161277771, median IoU: 0.7777708768844604, mAP = 0.496, mAP_50 = 0.92, mAP_75 = 0.469
TP: 1117, FN: 0, FP: 16
Precision: 0.9858781994704324
Recall: 1.0
Acc: 0.4519 Spec: 1.0000 Sens: 0.0000
Val Evaluation:
mIoU = 0.6334665417671204, median IoU: 0.6749613881111145,

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer2"
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Combination layer2, gbcnet VA, 50 epoch pretrained RN"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout = dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Combination layer2, gbcnet VA, 50 epoch pretrained RN/checkpoints/checkpoint_epoch_35.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 72.2MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 35, Start Epoch: 36, End Epoch: 100
VA Curriculum: False
Train Scores Length: 35, Val Scores Length: 35
Best accuracy so far: 0.7868852459016393

Epoch 36:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer2 and GBCNet backbones.
Train Loss: 0.6480759439014253, Total step: 21
Train Evaluation:
mIoU = 0.784231424331665, median IoU: 0.8018585443496704, mAP = 0.554, mAP_50 = 0.968, mAP_75 = 0.589
TP: 1127, FN: 0, FP: 6
Precision: 0.9947043248014121
Recall: 1.0
Acc: 0.8411 Spec: 0.9824 Sens: 0.7309
Val Evaluation:
mIoU = 0.6458122134208679, median IoU: 0.6878725290298462, mAP = 0.295, mAP_50 = 0.775, mAP_75 = 0.142
TP: 116, FN: 0, FP: 6
Precision: 0.9508196721311475
Recall: 1.0
Acc: 0.7623 Spec: 0.9625 Sens: 0.4762
Epoch time: 167.87799549102783
Estimated time remaining: 2:59:4
GPU Memory Used: 1207391744

Epoch

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer2"
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Combination layer2, gbcnet VA, 50 epoch pretrained RN"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout = dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Combination layer2, gbcnet VA, 50 epoch pretrained RN/checkpoints/checkpoint_epoch_85.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 113MB/s] 


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 85, Start Epoch: 86, End Epoch: 100
VA Curriculum: False
Train Scores Length: 85, Val Scores Length: 85
Best accuracy so far: 0.8852459016393442

Epoch 86:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer2 and GBCNet backbones.
Train Loss: 0.46323147274198984, Total step: 21
Train Evaluation:
mIoU = 0.8107105493545532, median IoU: 0.8253049850463867, mAP = 0.607, mAP_50 = 0.99, mAP_75 = 0.686
TP: 1128, FN: 0, FP: 5
Precision: 0.9955869373345102
Recall: 1.0
Acc: 0.8870 Spec: 0.9824 Sens: 0.8161
Val Evaluation:
mIoU = 0.6549793481826782, median IoU: 0.7006053924560547, mAP = 0.305, mAP_50 = 0.79, mAP_75 = 0.14
TP: 118, FN: 0, FP: 4
Precision: 0.9672131147540983
Recall: 1.0
Acc: 0.8279 Spec: 0.9875 Sens: 0.6190
Epoch time: 168.8791365623474
Estimated time remaining: 0:39:24
GPU Memory Used: 1207391744

Epoch 

### GBCRet, Combination layer3, gbcnet VA, 50 epoch pretrained RN

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer3"
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Combination layer3, gbcnet VA, 50 epoch pretrained RN"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=5, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 67.6MB/s]


Retinanet's COCO pre-trained weights are loaded.
Retinanet's 50 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 100
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0

Epoch 1:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer3 and GBCNet backbones.
Train Loss: 1.4334810234251476, Total step: 21
Train Evaluation:
mIoU = 0.7506561875343323, median IoU: 0.7778505086898804, mAP = 0.496, mAP_50 = 0.92, mAP_75 = 0.468
TP: 1117, FN: 0, FP: 16
Precision: 0.9858781994704324
Recall: 1.0
Acc: 0.4563 Spec: 1.0000 Sens: 0.0000
Val Evaluation:
mIoU = 0.6341177821159363, median IoU: 0.676551520824432

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer3"
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Combination layer3, gbcnet VA, 50 epoch pretrained RN"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout = dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Combination layer3, gbcnet VA, 50 epoch pretrained RN/checkpoints/checkpoint_epoch_95.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:03<00:00, 39.6MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 95, Start Epoch: 96, End Epoch: 100
VA Curriculum: False
Train Scores Length: 95, Val Scores Length: 95
Best accuracy so far: 0.8524590163934426

Epoch 96:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer3 and GBCNet backbones.
Train Loss: 0.4295916869526818, Total step: 21
Train Evaluation:
mIoU = 0.8139508366584778, median IoU: 0.8278469443321228, mAP = 0.614, mAP_50 = 0.99, mAP_75 = 0.721
TP: 1128, FN: 0, FP: 5
Precision: 0.9955869373345102
Recall: 1.0
Acc: 0.8976 Spec: 0.9890 Sens: 0.8475
Val Evaluation:
mIoU = 0.6544727683067322, median IoU: 0.6918175220489502, mAP = 0.299, mAP_50 = 0.778, mAP_75 = 0.17
TP: 119, FN: 0, FP: 3
Precision: 0.9754098360655737
Recall: 1.0
Acc: 0.8197 Spec: 0.9500 Sens: 0.6905
Epoch time: 185.55863690376282
Estimated time remaining: 0:12:22
GPU Memory Used: 1209253376

Epoch

### GBCRet, Combination layer4, gbcnet VA, 50 epoch pretrained RN

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Combination layer4, gbcnet VA, 50 epoch pretrained RN"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=5, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:03<00:00, 42.7MB/s]


Retinanet's COCO pre-trained weights are loaded.
Retinanet's 50 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 100
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0

Epoch 1:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Train Loss: 1.4427331742786227, Total step: 21
Train Evaluation:
mIoU = 0.7506058812141418, median IoU: 0.7779982089996338, mAP = 0.496, mAP_50 = 0.92, mAP_75 = 0.469
TP: 1117, FN: 0, FP: 16
Precision: 0.9858781994704324
Recall: 1.0
Acc: 0.4722 Spec: 0.9451 Sens: 0.1659
Val Evaluation:
mIoU = 0.6336363554000854, median IoU: 0.674967944622039

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Combination layer4, gbcnet VA, 50 epoch pretrained RN"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout = dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Combination layer4, gbcnet VA, 50 epoch pretrained RN/checkpoints/checkpoint_epoch_30.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            num_worse_epochs=num_worse_epochs, patience=50)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 72.4MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 30, Start Epoch: 31, End Epoch: 100
VA Curriculum: False
Train Scores Length: 30, Val Scores Length: 30
Best accuracy so far: 0.7459016393442623
Patience so far: 0

Epoch 31:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Train Loss: 0.7234791488874526, Total step: 21
Train Evaluation:
mIoU = 0.7764686346054077, median IoU: 0.8009214401245117, mAP = 0.542, mAP_50 = 0.957, mAP_75 = 0.551
TP: 1123, FN: 0, FP: 10
Precision: 0.9911738746690203
Recall: 1.0
Acc: 0.8138 Spec: 0.9846 Sens: 0.6457
Val Evaluation:
mIoU = 0.6573623418807983, median IoU: 0.7105692625045776, mAP = 0.307, mAP_50 = 0.779, mAP_75 = 0.155
TP: 118, FN: 0, FP: 4
Precision: 0.9672131147540983
Recall: 1.0
Acc: 0.7787 Spec: 0.9750 Sens: 0.5476
Epoch time: 161.61501908302307
Estimated time remaining: 3:5:51
GPU Memory Us

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Combination layer4, gbcnet VA, 50 epoch pretrained RN"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout = dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Combination layer4, gbcnet VA, 50 epoch pretrained RN/checkpoints/checkpoint_epoch_95.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            num_worse_epochs=num_worse_epochs, patience=50)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 84.9MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 95, Start Epoch: 96, End Epoch: 200
VA Curriculum: False
Train Scores Length: 95, Val Scores Length: 95
Best accuracy so far: 0.8770491803278688
Patience so far: 20

Epoch 96:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Train Loss: 0.4376900735355559, Total step: 21
Train Evaluation:
mIoU = 0.813261866569519, median IoU: 0.8260235786437988, mAP = 0.613, mAP_50 = 0.99, mAP_75 = 0.725
TP: 1128, FN: 0, FP: 5
Precision: 0.9955869373345102
Recall: 1.0
Acc: 0.8914 Spec: 0.9835 Sens: 0.8475
Val Evaluation:
mIoU = 0.6597842574119568, median IoU: 0.6889591813087463, mAP = 0.306, mAP_50 = 0.806, mAP_75 = 0.153
TP: 118, FN: 0, FP: 4
Precision: 0.9672131147540983
Recall: 1.0
Acc: 0.8361 Spec: 0.9625 Sens: 0.6667
Epoch time: 172.7938666343689
Estimated time remaining: 4:59:31
GPU Memory Used

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Combination layer4, gbcnet VA, 50 epoch pretrained RN"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout = dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Combination layer4, gbcnet VA, 50 epoch pretrained RN/checkpoints/checkpoint_epoch_126.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 92.4MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 126, Start Epoch: 127, End Epoch: 200
VA Curriculum: False
Train Scores Length: 126, Val Scores Length: 126
Best accuracy so far: 0.8770491803278688
Patience so far: 0

Epoch 127:
Number of Train images: 1133
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Train Loss: 0.3731378047239213, Total step: 21
Train Evaluation:
mIoU = 0.8256927728652954, median IoU: 0.8379100561141968, mAP = 0.643, mAP_50 = 0.99, mAP_75 = 0.769
TP: 1129, FN: 0, FP: 4
Precision: 0.9964695498676082
Recall: 1.0
Acc: 0.9135 Spec: 0.9857 Sens: 0.8655
Val Evaluation:
mIoU = 0.6590274572372437, median IoU: 0.6972386837005615, mAP = 0.316, mAP_50 = 0.794, mAP_75 = 0.199
TP: 118, FN: 0, FP: 4
Precision: 0.9672131147540983
Recall: 1.0
Acc: 0.8361 Spec: 0.9875 Sens: 0.6190
Epoch time: 166.65722155570984
Estimated time remaining: 3:22:46
GPU Memor

KeyboardInterrupt: 

### GBCRet, gbcnet VA, 50 epochs pretrained RN, Hierarchical classifiation, criterion = nn.BCEWithLogitsLoss()

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="None"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN"

criterion = nn.BCEWithLogitsLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn, do_hierarchical_cls=do_hierarchical_cls,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=5, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 150MB/s]


Retinanet's COCO pre-trained weights are loaded.
Retinanet's 50 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 100
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0

Epoch 1:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Hierarchical Classification is Activated.
Train Loss: 1.7456396473081488, Total step: 38
Train Evaluation:
mIoU = 0.7458564639091492, median IoU: 0.7719936966896057, mAP = 0.479, mAP_50 = 0.931, mAP_75 = 0.419
TP: 2005, FN: 0, FP: 31
Precision: 0.9847740667976425
Recall: 1.0
Acc: 0.2937 Spec: 0.8900 Sens: 0.1965
Val Evaluation:
mIoU = 0.6157771348953247, median IoU: 0.65045565366745, mA

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="None"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN"

criterion = nn.BCEWithLogitsLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN/checkpoints/checkpoint_epoch_15.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:09<00:00, 16.6MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 15, Start Epoch: 16, End Epoch: 100
VA Curriculum: False
Train Scores Length: 15, Val Scores Length: 15
Best accuracy so far: 0.5901639344262295

Epoch 16:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Hierarchical Classification is Activated.
Train Loss: 1.250996133214549, Total step: 38
Train Evaluation:
mIoU = 0.7741162180900574, median IoU: 0.7947160601615906, mAP = 0.532, mAP_50 = 0.967, mAP_75 = 0.53
TP: 2021, FN: 0, FP: 15
Precision: 0.9926326129666012
Recall: 1.0
Acc: 0.6793 Spec: 0.7629 Sens: 0.9450
Val Evaluation:
mIoU = 0.6389946341514587, median IoU: 0.6617689728736877, mAP = 0.283, mAP_50 = 0.764, mAP_75 = 0.137
TP: 116, FN: 0, FP: 6
Precision: 0.9508196721311475
Recall: 1.0
Acc: 0.6557 Spec: 0.5875 Sens: 0.9286
Epoch time: 284.7300064563751
Estimated time remaining: 6:38:37
GPU Memory Used: 1541628928

Epoch 17:

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="None"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN"

criterion = nn.BCEWithLogitsLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN/checkpoints/checkpoint_epoch_33.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:05<00:00, 26.5MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 33, Start Epoch: 34, End Epoch: 100
VA Curriculum: False
Train Scores Length: 33, Val Scores Length: 33
Best accuracy so far: 0.8688524590163934

Epoch 34:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Hierarchical Classification is Activated.
Train Loss: 0.7199722873537164, Total step: 38
Train Evaluation:
mIoU = 0.7933767437934875, median IoU: 0.8112497925758362, mAP = 0.57, mAP_50 = 0.979, mAP_75 = 0.607
TP: 2023, FN: 0, FP: 13
Precision: 0.993614931237721
Recall: 1.0
Acc: 0.8831 Spec: 0.9614 Sens: 0.8998
Val Evaluation:
mIoU = 0.6460070610046387, median IoU: 0.6819339990615845, mAP = 0.29, mAP_50 = 0.779, mAP_75 = 0.153
TP: 117, FN: 0, FP: 5
Precision: 0.9590163934426229
Recall: 1.0
Acc: 0.8607 Spec: 0.8875 Sens: 0.8571
Epoch time: 301.0213568210602
Estimated time remaining: 5:31:7
GPU Memory Used: 1179613696

Epoch 35:
T

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="None"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN"

criterion = nn.BCEWithLogitsLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN/checkpoints/checkpoint_epoch_48.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:03<00:00, 49.8MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 48, Start Epoch: 49, End Epoch: 100
VA Curriculum: False
Train Scores Length: 48, Val Scores Length: 48
Best accuracy so far: 0.8770491803278688

Epoch 49:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Hierarchical Classification is Activated.
Train Loss: 0.5961451365759498, Total step: 38
Train Evaluation:
mIoU = 0.8064119815826416, median IoU: 0.8213831782341003, mAP = 0.599, mAP_50 = 0.98, mAP_75 = 0.681
TP: 2025, FN: 0, FP: 11
Precision: 0.9945972495088409
Recall: 1.0
Acc: 0.9018 Spec: 0.9764 Sens: 0.8723
Val Evaluation:
mIoU = 0.6499919891357422, median IoU: 0.6859638094902039, mAP = 0.305, mAP_50 = 0.813, mAP_75 = 0.152
TP: 116, FN: 0, FP: 6
Precision: 0.9508196721311475
Recall: 1.0
Acc: 0.8770 Spec: 0.9500 Sens: 0.7857
Epoch time: 280.100394487381
Estimated time remaining: 3:58:5
GPU Memory Used: 1179613696

Epoch 50:


In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="None"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN"

criterion = nn.BCEWithLogitsLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN/checkpoints/checkpoint_epoch_75.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 61.7MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 75, Start Epoch: 76, End Epoch: 100
VA Curriculum: False
Train Scores Length: 75, Val Scores Length: 75
Best accuracy so far: 0.8770491803278688

Epoch 76:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Hierarchical Classification is Activated.
Train Loss: 0.4680395251826236, Total step: 38
Train Evaluation:
mIoU = 0.8214496374130249, median IoU: 0.8343948125839233, mAP = 0.637, mAP_50 = 0.99, mAP_75 = 0.784
TP: 2027, FN: 0, FP: 9
Precision: 0.9955795677799607
Recall: 1.0
Acc: 0.9234 Spec: 0.9764 Sens: 0.9470
Val Evaluation:
mIoU = 0.637865424156189, median IoU: 0.6743992567062378, mAP = 0.285, mAP_50 = 0.764, mAP_75 = 0.165
TP: 116, FN: 0, FP: 6
Precision: 0.9508196721311475
Recall: 1.0
Acc: 0.8443 Spec: 0.8875 Sens: 0.8333
Epoch time: 293.15590357780457
Estimated time remaining: 1:57:16
GPU Memory Used: 1179613696

Epoch 77:

### GBCRet, gbcnet VA, 50 epochs pretrained RN, Hierarchical classifiation, criterion = nn.CrossEntropyLoss()

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="None"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn, do_hierarchical_cls=do_hierarchical_cls,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=5, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 59.9MB/s]


Retinanet's COCO pre-trained weights are loaded.
Retinanet's 50 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 100
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0
Patience so far: 0

Epoch 1:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Hierarchical Classification is Activated.
Train Loss: 1.7360350332762067, Total step: 38
Train Evaluation:
mIoU = 0.7458552122116089, median IoU: 0.7721089124679565, mAP = 0.479, mAP_50 = 0.931, mAP_75 = 0.419
TP: 2005, FN: 0, FP: 31
Precision: 0.9847740667976425
Recall: 1.0
Acc: 0.3188 Spec: 0.7754 Sens: 0.3556
Val Evaluation:
mIoU = 0.6157749891281128, median IoU: 0

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="None"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss/checkpoints/checkpoint_epoch_15.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=50)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 52.9MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 15, Start Epoch: 16, End Epoch: 100
VA Curriculum: False
Train Scores Length: 15, Val Scores Length: 15
Best accuracy so far: 0.5573770491803278
Patience so far: 0

Epoch 16:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Hierarchical Classification is Activated.
Train Loss: 0.7809320170628397, Total step: 38
Train Evaluation:
mIoU = 0.7740709185600281, median IoU: 0.795211672782898, mAP = 0.532, mAP_50 = 0.967, mAP_75 = 0.518
TP: 2022, FN: 0, FP: 14
Precision: 0.9931237721021611
Recall: 1.0
Acc: 0.6714 Spec: 0.7485 Sens: 0.9568
Val Evaluation:
mIoU = 0.6401868462562561, median IoU: 0.6708438992500305, mAP = 0.287, mAP_50 = 0.762, mAP_75 = 0.151
TP: 116, FN: 0, FP: 6
Precision: 0.9508196721311475
Recall: 1.0
Acc: 0.6557 Spec: 0.6000 Sens: 0.9048
Epoch time: 285.00210666656494
Estimated time remaining: 6:39:0
GPU Memory Used: 1

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="None"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss/checkpoints/checkpoint_epoch_54.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=50)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 83.6MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 54, Start Epoch: 55, End Epoch: 100
VA Curriculum: False
Train Scores Length: 54, Val Scores Length: 54
Best accuracy so far: 0.8852459016393442
Patience so far: 11

Epoch 55:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Hierarchical Classification is Activated.
Train Loss: 0.48667079445562866, Total step: 38
Train Evaluation:
mIoU = 0.811843752861023, median IoU: 0.8257032036781311, mAP = 0.611, mAP_50 = 0.99, mAP_75 = 0.706
TP: 2028, FN: 0, FP: 8
Precision: 0.9960707269155207
Recall: 1.0
Acc: 0.9224 Spec: 0.9777 Sens: 0.9214
Val Evaluation:
mIoU = 0.6494231224060059, median IoU: 0.6754351854324341, mAP = 0.296, mAP_50 = 0.777, mAP_75 = 0.148
TP: 119, FN: 0, FP: 3
Precision: 0.9754098360655737
Recall: 1.0
Acc: 0.8361 Spec: 0.9000 Sens: 0.7619
Epoch time: 300.99563360214233
Estimated time remaining: 3:45:45
GPU Memory Used: 

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="None"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss/checkpoints/checkpoint_epoch_72.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=50)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 152MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 72, Start Epoch: 73, End Epoch: 200
VA Curriculum: False
Train Scores Length: 72, Val Scores Length: 72
Best accuracy so far: 0.8852459016393442
Patience so far: 29

Epoch 73:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Hierarchical Classification is Activated.
Train Loss: 0.44824859656785665, Total step: 38
Train Evaluation:
mIoU = 0.8224861025810242, median IoU: 0.834536075592041, mAP = 0.635, mAP_50 = 0.99, mAP_75 = 0.768
TP: 2028, FN: 0, FP: 8
Precision: 0.9960707269155207
Recall: 1.0
Acc: 0.9352 Spec: 0.9843 Sens: 0.9430
Val Evaluation:
mIoU = 0.6540899276733398, median IoU: 0.6800422072410583, mAP = 0.302, mAP_50 = 0.797, mAP_75 = 0.136
TP: 116, FN: 0, FP: 6
Precision: 0.9508196721311475
Recall: 1.0
Acc: 0.8525 Spec: 0.9000 Sens: 0.7857
Epoch time: 307.9499053955078
Estimated time remaining: 10:51:50
GPU Memory Used: 

### GBCRet, layer4 combination, gbcnet VA, 50 epochs pretrained RN, Hierarchical classifiation, criterion = nn.CrossEntropyLoss()

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn, do_hierarchical_cls=do_hierarchical_cls,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=5, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler, patience=50)

Retinanet's COCO pre-trained weights are loaded.
Retinanet's 50 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 100
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0
Patience so far: 0

Epoch 1:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 1.7430117475359064, Total step: 38
Train Evaluation:
mIoU = 0.7502132058143616, median IoU: 0.7794556617736816, mAP = 0.488, mAP_50 = 0.932, mAP_75 = 0.436
TP: 2007, FN: 0, FP: 29
Precision: 0.9857563850687623
Recall: 1.0
Acc: 0.5324 Spec: 0.6248 Sens: 0.6051
Val Evalua

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss/checkpoints/checkpoint_epoch_15.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=50)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:03<00:00, 46.5MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 15, Start Epoch: 16, End Epoch: 100
VA Curriculum: False
Train Scores Length: 15, Val Scores Length: 15
Best accuracy so far: 0.5081967213114754
Patience so far: 3

Epoch 16:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.8065944408115587, Total step: 38
Train Evaluation:
mIoU = 0.775057315826416, median IoU: 0.7948744297027588, mAP = 0.532, mAP_50 = 0.967, mAP_75 = 0.519
TP: 2019, FN: 0, FP: 17
Precision: 0.9916502946954814
Recall: 1.0
Acc: 0.7279 Spec: 0.8199 Sens: 0.9077
Val Evaluation:
mIoU = 0.6408363580703735, median IoU: 0.6719692945480347, mAP = 0.279, mAP_50 = 0.703, mAP_75 = 0.148
TP: 117, FN: 0, FP: 5
Precision: 0.9590163934426229
Recall: 1.0
Acc: 0.6967 Spec: 0.6875 Sens: 0.9048
Epoch time: 292.28009390830994
Estim

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.4
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss/checkpoints/checkpoint_epoch_66.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=50)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:03<00:00, 40.0MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 66, Start Epoch: 67, End Epoch: 200
VA Curriculum: False
Train Scores Length: 66, Val Scores Length: 66
Best accuracy so far: 0.9098360655737705
Patience so far: 22

Epoch 67:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.4276905914670543, Total step: 38
Train Evaluation:
mIoU = 0.8097653388977051, median IoU: 0.8222042322158813, mAP = 0.61, mAP_50 = 0.99, mAP_75 = 0.712
TP: 2026, FN: 0, FP: 10
Precision: 0.9950884086444007
Recall: 1.0
Acc: 0.9411 Spec: 0.9856 Sens: 0.9607
Val Evaluation:
mIoU = 0.654601514339447, median IoU: 0.6998497247695923, mAP = 0.3, mAP_50 = 0.775, mAP_75 = 0.156
TP: 119, FN: 0, FP: 3
Precision: 0.9754098360655737
Recall: 1.0
Acc: 0.8689 Spec: 0.9625 Sens: 0.7381
Epoch time: 287.7362365722656
Estimated

### GBCRet, layer4 combination, gbcnet VA, 50 epochs pretrained RN, Hierarchical classifiation, criterion = nn.CrossEntropyLoss(), dropout=0.5

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn, do_hierarchical_cls=do_hierarchical_cls,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=5, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler, patience=50)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 142MB/s]


Retinanet's COCO pre-trained weights are loaded.
Retinanet's 50 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 100
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0
Patience so far: 0

Epoch 1:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 1.7434121872249402, Total step: 38
Train Evaluation:
mIoU = 0.7501730918884277, median IoU: 0.7791094779968262, mAP = 0.488, mAP_50 = 0.932, mAP_75 = 0.437
TP: 2007, FN: 0, FP: 29
Precision: 0.9857563850687623
Recall: 1.0
Acc: 0.5398 Spec: 0.6320 Sens: 0.6169
Val Evalua

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.5
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5/checkpoints/checkpoint_epoch_35.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=50)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 71.7MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 35, Start Epoch: 36, End Epoch: 200
VA Curriculum: False
Train Scores Length: 35, Val Scores Length: 35
Best accuracy so far: 0.9098360655737705
Patience so far: 0

Epoch 36:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.5555684809622011, Total step: 38
Train Evaluation:
mIoU = 0.7965930104255676, median IoU: 0.8130502104759216, mAP = 0.575, mAP_50 = 0.979, mAP_75 = 0.636
TP: 2024, FN: 0, FP: 12
Precision: 0.9941060903732809
Recall: 1.0
Acc: 0.8885 Spec: 0.9653 Sens: 0.8841
Val Evaluation:
mIoU = 0.6447915434837341, median IoU: 0.685694694519043, mAP = 0.285, mAP_50 = 0.751, mAP_75 = 0.153
TP: 118, FN: 0, FP: 4
Precision: 0.9672131147540983
Recall: 1.0
Acc: 0.8770 Spec: 0.9125 Sens: 0.8810

Checkpoint is saved to: /content/dr

KeyboardInterrupt: 

### GBCRet, layer4 combination, gbcnet VA, 50 epochs pretrained RN, Hierarchical classifiation, criterion = nn.CrossEntropyLoss(), dropout=0.6

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.6
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.6"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn, do_hierarchical_cls=do_hierarchical_cls,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=3, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 99.3MB/s]


Retinanet's COCO pre-trained weights are loaded.
Retinanet's 50 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 100
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0
Patience so far: 0

Epoch 1:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 1.7435254517354464, Total step: 38
Train Evaluation:
mIoU = 0.7501829862594604, median IoU: 0.7790377736091614, mAP = 0.489, mAP_50 = 0.932, mAP_75 = 0.436
TP: 2007, FN: 0, FP: 29
Precision: 0.9857563850687623
Recall: 1.0
Acc: 0.5393 Spec: 0.6339 Sens: 0.6169
 Normal vs

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.6
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.6"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.6/checkpoints/checkpoint_epoch_45.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 147MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 45, Start Epoch: 46, End Epoch: 200
VA Curriculum: False
Train Scores Length: 45, Val Scores Length: 45
Best accuracy so far: 0.9098360655737705
Patience so far: 10

Epoch 46:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.5354620273175993, Total step: 38
Train Evaluation:
mIoU = 0.80145663022995, median IoU: 0.8170486688613892, mAP = 0.59, mAP_50 = 0.98, mAP_75 = 0.671
TP: 2024, FN: 0, FP: 12
Precision: 0.9941060903732809
Recall: 1.0
Acc: 0.9062 Spec: 0.9784 Sens: 0.8978
Normal vs Not Noraml Acc: 0.9357 Benign vs Malignant Acc: 0.9351
Val Evaluation:
mIoU = 0.6508463025093079, median IoU: 0.6941805481910706, mAP = 0.294, mAP_50 = 0.785, mAP_75 = 0.163
TP: 115, FN: 0, FP: 7
Precision: 0.9426229508196722
Recall: 1.0
Acc: 0.8279

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 200
batch_size = 55
dropout = 0.6
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.6"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.6/checkpoints/checkpoint_epoch_63.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:03<00:00, 44.9MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 63, Start Epoch: 64, End Epoch: 200
VA Curriculum: False
Train Scores Length: 63, Val Scores Length: 63
Best accuracy so far: 0.9098360655737705
Patience so far: 28

Epoch 64:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.44051766709277507, Total step: 38
Train Evaluation:
mIoU = 0.812056303024292, median IoU: 0.8251294493675232, mAP = 0.609, mAP_50 = 0.99, mAP_75 = 0.711
TP: 2027, FN: 0, FP: 9
Precision: 0.9955795677799607
Recall: 1.0
Acc: 0.9479 Spec: 0.9902 Sens: 0.9548
Normal vs Not Noraml Acc: 0.9641 Benign vs Malignant Acc: 0.9659
Val Evaluation:
mIoU = 0.6550950407981873, median IoU: 0.6984061002731323, mAP = 0.301, mAP_50 = 0.784, mAP_75 = 0.145
TP: 116, FN: 0, FP: 6
Precision: 0.9508196721311475
Recall: 1.0
Acc: 0.90

### GBCRet, layer4 combination, gbcnet VA, 50 epochs pretrained RN, Hierarchical classifiation, criterion = nn.CrossEntropyLoss(), dropout=0.5, with attention

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = False # Apply attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, with attention"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn, do_hierarchical_cls=do_hierarchical_cls,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=3, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 76.6MB/s]


Retinanet's COCO pre-trained weights are loaded.
Retinanet's 50 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 100
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0
Patience so far: 0

Epoch 1:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Self attention is applied on combined features.
Hierarchical Classification is Activated.
Train Loss: 1.7484924259938692, Total step: 38
Train Evaluation:
mIoU = 0.7501423358917236, median IoU: 0.779209554195404, mAP = 0.488, mAP_50 = 0.932, mAP_75 = 0.436
TP: 2007, FN: 0, FP: 29
Precision: 0.9857563850687623
Recall: 1.0
A

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = False # Apply attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, with attention"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, with attention/checkpoints/checkpoint_epoch_6.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 79.9MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 6, Start Epoch: 7, End Epoch: 100
VA Curriculum: False
Train Scores Length: 6, Val Scores Length: 6
Best accuracy so far: 0.45901639344262296
Patience so far: 0

Epoch 7:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Self attention is applied on combined features.
Hierarchical Classification is Activated.
Train Loss: 1.6452582164814598, Total step: 38
Train Evaluation:
mIoU = 0.7608153223991394, median IoU: 0.7829052209854126, mAP = 0.505, mAP_50 = 0.953, mAP_75 = 0.459
TP: 2015, FN: 0, FP: 21
Precision: 0.9896856581532416
Recall: 1.0
Acc: 0.4239 Spec: 0.5331 Sens: 0.7583
Normal vs Not Noraml Acc: 0.5982 Benign vs Malignant Acc: 0.6450
Val Evaluation:
mIoU = 0.6211196184158325, median IoU: 0.6462176442146301, mAP = 0.266, mAP_50 = 0.706, mAP_75 = 0.152
TP: 115, FN: 0, FP: 7
Precis

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = False # Apply attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, with attention"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, with attention/checkpoints/checkpoint_epoch_57.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 56.8MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 57, Start Epoch: 58, End Epoch: 100
VA Curriculum: False
Train Scores Length: 57, Val Scores Length: 57
Best accuracy so far: 0.9180327868852459
Patience so far: 22

Epoch 58:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Self attention is applied on combined features.
Hierarchical Classification is Activated.
Train Loss: 0.42879808733337804, Total step: 38
Train Evaluation:
mIoU = 0.8044531941413879, median IoU: 0.8184390664100647, mAP = 0.599, mAP_50 = 0.98, mAP_75 = 0.678
TP: 2026, FN: 0, FP: 10
Precision: 0.9950884086444007
Recall: 1.0
Acc: 0.9499 Spec: 0.9836 Sens: 0.9862
Normal vs Not Noraml Acc: 0.9627 Benign vs Malignant Acc: 0.9733
Val Evaluation:
mIoU = 0.6251574754714966, median IoU: 0.6640004515647888, mAP = 0.268, mAP_50 = 0.735, mAP_75 = 0.129
TP: 111, FN: 0, FP: 11


### GBCRet, layer4 combination, gbcnet VA, 50 epochs pretrained RN, Hierarchical classifiation, criterion = nn.CrossEntropyLoss(), dropout=0.5, bm_fc_layers="2"

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="2"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=2"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn, do_hierarchical_cls=do_hierarchical_cls,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet,
                         bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=3, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:03<00:00, 49.3MB/s]


Retinanet's COCO pre-trained weights are loaded.
Retinanet's 50 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 100
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0
Patience so far: 0

Epoch 1:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 1.7212328942198503, Total step: 38
Train Evaluation:
mIoU = 0.7453113794326782, median IoU: 0.7786130905151367, mAP = 0.484, mAP_50 = 0.931, mAP_75 = 0.447
TP: 1994, FN: 0, FP: 42
Precision: 0.9793713163064833
Recall: 1.0
Acc: 0.4862 Spec: 1.0000 Sens: 0.0020
Normal vs 

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="2"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=2"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=2/checkpoints/checkpoint_epoch_39.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 65.6MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 39, Start Epoch: 40, End Epoch: 100
VA Curriculum: False
Train Scores Length: 39, Val Scores Length: 39
Best accuracy so far: 0.8934426229508197
Patience so far: 14

Epoch 40:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.597415062942003, Total step: 38
Train Evaluation:
mIoU = 0.7996951341629028, median IoU: 0.8142871856689453, mAP = 0.587, mAP_50 = 0.98, mAP_75 = 0.653
TP: 2026, FN: 0, FP: 10
Precision: 0.9950884086444007
Recall: 1.0
Acc: 0.8988 Spec: 0.9699 Sens: 0.8900
Normal vs Not Noraml Acc: 0.9347 Benign vs Malignant Acc: 0.9209
Val Evaluation:
mIoU = 0.6444278955459595, median IoU: 0.6806510090827942, mAP = 0.284, mAP_50 = 0.762, mAP_75 = 0.152
TP: 115, FN: 0, FP: 7
Precision: 0.9426229508196722
Recall: 1.0
Acc: 0.89

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="2"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=2"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=2/checkpoints/checkpoint_epoch_63.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 75.5MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 63, Start Epoch: 64, End Epoch: 100
VA Curriculum: False
Train Scores Length: 63, Val Scores Length: 63
Best accuracy so far: 0.9016393442622951
Patience so far: 22

Epoch 64:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.4232287053999148, Total step: 38
Train Evaluation:
mIoU = 0.8118540644645691, median IoU: 0.8244959115982056, mAP = 0.61, mAP_50 = 0.99, mAP_75 = 0.711
TP: 2025, FN: 0, FP: 11
Precision: 0.9945972495088409
Recall: 1.0
Acc: 0.9381 Spec: 0.9915 Sens: 0.9430
Normal vs Not Noraml Acc: 0.9558 Benign vs Malignant Acc: 0.9620
Val Evaluation:
mIoU = 0.6372005939483643, median IoU: 0.6779190897941589, mAP = 0.276, mAP_50 = 0.735, mAP_75 = 0.143
TP: 116, FN: 0, FP: 6
Precision: 0.9508196721311475
Recall: 1.0
Acc: 0.86

### GBCRet, layer4 combination, gbcnet VA, 50 epochs pretrained RN, Hierarchical classifiation, criterion = nn.CrossEntropyLoss(), dropout=0.5, bm_fc_layers="3"

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn, do_hierarchical_cls=do_hierarchical_cls,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet,
                         bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=3, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 57.8MB/s]


Retinanet's COCO pre-trained weights are loaded.
Retinanet's 50 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 100
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0
Patience so far: 0

Epoch 1:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 1.7355056185471385, Total step: 38
Train Evaluation:
mIoU = 0.7434861660003662, median IoU: 0.7773441076278687, mAP = 0.475, mAP_50 = 0.921, mAP_75 = 0.428
TP: 2002, FN: 0, FP: 34
Precision: 0.9833005893909627
Recall: 1.0
Acc: 0.5064 Spec: 0.7904 Sens: 0.4872
Normal vs 

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_9.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 81.3MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 9, Start Epoch: 10, End Epoch: 100
VA Curriculum: False
Train Scores Length: 9, Val Scores Length: 9
Best accuracy so far: 0.5655737704918032
Patience so far: 8

Epoch 10:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 1.3352379767518294, Total step: 38
Train Evaluation:
mIoU = 0.7622086405754089, median IoU: 0.7838467955589294, mAP = 0.511, mAP_50 = 0.956, mAP_75 = 0.484
TP: 2013, FN: 0, FP: 23
Precision: 0.9887033398821218
Recall: 1.0
Acc: 0.3806 Spec: 0.5462 Sens: 0.7485
Normal vs Not Noraml Acc: 0.5491 Benign vs Malignant Acc: 0.6621
Val Evaluation:
mIoU = 0.6344945430755615, median IoU: 0.6730465292930603, mAP = 0.289, mAP_50 = 0.744, mAP_75 = 0.171
TP: 115, FN: 0, FP: 7
Precision: 0.9426229508196722
Recall: 1.0
Acc: 0.4590

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_27.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 129MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 27, Start Epoch: 28, End Epoch: 100
VA Curriculum: False
Train Scores Length: 27, Val Scores Length: 27
Best accuracy so far: 0.8934426229508197
Patience so far: 0

Epoch 28:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.6485337348360765, Total step: 38
Train Evaluation:
mIoU = 0.7883122563362122, median IoU: 0.807066023349762, mAP = 0.562, mAP_50 = 0.979, mAP_75 = 0.602
TP: 2021, FN: 0, FP: 15
Precision: 0.9926326129666012
Recall: 1.0
Acc: 0.8846 Spec: 0.9731 Sens: 0.8625
Normal vs Not Noraml Acc: 0.9185 Benign vs Malignant Acc: 0.9226
Val Evaluation:
mIoU = 0.6514549255371094, median IoU: 0.686207115650177, mAP = 0.3, mAP_50 = 0.792, mAP_75 = 0.129
TP: 117, FN: 0, FP: 5
Precision: 0.9590163934426229
Recall: 1.0
Acc: 0.8770 

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_45.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:09<00:00, 15.7MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 45, Start Epoch: 46, End Epoch: 100
VA Curriculum: False
Train Scores Length: 45, Val Scores Length: 45
Best accuracy so far: 0.9098360655737705
Patience so far: 3

Epoch 46:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.5089327324377863, Total step: 38
Train Evaluation:
mIoU = 0.8029489517211914, median IoU: 0.8165514469146729, mAP = 0.595, mAP_50 = 0.98, mAP_75 = 0.681
TP: 2023, FN: 0, FP: 13
Precision: 0.993614931237721
Recall: 1.0
Acc: 0.9140 Spec: 0.9777 Sens: 0.9194
Normal vs Not Noraml Acc: 0.9401 Benign vs Malignant Acc: 0.9423
Val Evaluation:
mIoU = 0.6421762108802795, median IoU: 0.6740090847015381, mAP = 0.286, mAP_50 = 0.769, mAP_75 = 0.17
TP: 115, FN: 0, FP: 7
Precision: 0.9426229508196722
Recall: 1.0
Acc: 0.8689

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="3"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=3/checkpoints/checkpoint_epoch_54.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 137MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 54, Start Epoch: 55, End Epoch: 100
VA Curriculum: False
Train Scores Length: 54, Val Scores Length: 54
Best accuracy so far: 0.9262295081967213
Patience so far: 3

Epoch 55:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.4342935579387765, Total step: 38
Train Evaluation:
mIoU = 0.809762716293335, median IoU: 0.8256258964538574, mAP = 0.607, mAP_50 = 0.98, mAP_75 = 0.699
TP: 2024, FN: 0, FP: 12
Precision: 0.9941060903732809
Recall: 1.0
Acc: 0.9317 Spec: 0.9843 Sens: 0.9430
Normal vs Not Noraml Acc: 0.9528 Benign vs Malignant Acc: 0.9547
Val Evaluation:
mIoU = 0.6389867663383484, median IoU: 0.6788204908370972, mAP = 0.285, mAP_50 = 0.752, mAP_75 = 0.161
TP: 115, FN: 0, FP: 7
Precision: 0.9426229508196722
Recall: 1.0
Acc: 0.860

### GBCRet, layer4 combination, gbcnet VA, 50 epochs pretrained RN, Hierarchical classifiation, criterion = nn.CrossEntropyLoss(), dropout=0.5, bm_fc_layers="4"

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="4"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)

load_pretrained_retinanet=True
retinanet_weight_path = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth"
gbcnet_weight_path = "/content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth"
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=4"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(combined=combined, criterion=criterion, gbcnet_weight_path=gbcnet_weight_path, retinanet_weight_path=retinanet_weight_path,
                         gold_curriculum=gold_curriculum, retinanet_backbone=retinanet_backbone, width=width, height=height,
                         min_size=min_size, max_size=max_size, dropout=dropout, no_attn=no_attn, do_hierarchical_cls=do_hierarchical_cls,
                         gbcnet_va_curriculum=gbcnet_va_curriculum, load_pretrained_retinanet=load_pretrained_retinanet,
                         bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, train_transform=train_transform, eval_transform=eval_transform,
            checkpoint_save_rate=3, va_curriculum=va_curriculum, batch_size=batch_size, save_model_dir=save_model_dir,
            lr_scheduler=lr_scheduler, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:02<00:00, 60.8MB/s]


Retinanet's COCO pre-trained weights are loaded.
Retinanet's 50 epoch Pretrained Weights from: /content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/RN, COCO/checkpoints/checkpoint_epoch_50.pth
GBCNet's pretrained weights are loaded from: /content/drive/MyDrive/CV/Project/Code/Original Models/init_weights.pth
Last Epoch: 0, Start Epoch: 1, End Epoch: 100
VA Curriculum: False
Train Scores Length: 0, Val Scores Length: 0
Best accuracy so far: 0
Patience so far: 0

Epoch 1:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 1.7414831738722951, Total step: 38
Train Evaluation:
mIoU = 0.7492842674255371, median IoU: 0.7758486270904541, mAP = 0.487, mAP_50 = 0.933, mAP_75 = 0.435
TP: 2010, FN: 0, FP: 26
Precision: 0.9872298624754421
Recall: 1.0
Acc: 0.4990 Spec: 0.8644 Sens: 0.3261
Normal vs 

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="4"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=4"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=4/checkpoints/checkpoint_epoch_33.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:01<00:00, 129MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 33, Start Epoch: 34, End Epoch: 100
VA Curriculum: False
Train Scores Length: 33, Val Scores Length: 33
Best accuracy so far: 0.8770491803278688
Patience so far: 4

Epoch 34:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.5746229141950607, Total step: 38
Train Evaluation:
mIoU = 0.7942032814025879, median IoU: 0.8102601766586304, mAP = 0.577, mAP_50 = 0.979, mAP_75 = 0.639
TP: 2023, FN: 0, FP: 13
Precision: 0.993614931237721
Recall: 1.0
Acc: 0.8787 Spec: 0.9587 Sens: 0.8919
Normal vs Not Noraml Acc: 0.9224 Benign vs Malignant Acc: 0.9069
Val Evaluation:
mIoU = 0.6481016278266907, median IoU: 0.6853523254394531, mAP = 0.295, mAP_50 = 0.783, mAP_75 = 0.162
TP: 116, FN: 0, FP: 6
Precision: 0.9508196721311475
Recall: 1.0
Acc: 0.83

In [ ]:
lr = 0.001
weight_decay = 0.001
momentum = 0.9
width = 224
height = 224
min_size = 224
max_size = 224
va_curriculum = False
gbcnet_va_curriculum = True
gold_curriculum = False
no_attn = True # No attention
combined="resnet_layer4"
do_hierarchical_cls=True
retinanet_backbone="resnet50-coco"
num_epochs = 100
batch_size = 55
dropout = 0.5
bm_fc_layers="4"
train_transform = get_train_transforms(width, height)
eval_transform = get_eval_transforms(width, height)
save_model_dir = "/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=4"

criterion = nn.CrossEntropyLoss()
gbcret = GBCRet_combined(continue_training=True, combined=combined, criterion=criterion, gold_curriculum=gold_curriculum,
                         retinanet_backbone=retinanet_backbone, width=width, height=height, min_size=min_size, max_size=max_size,
                         dropout=dropout, no_attn=no_attn, gbcnet_va_curriculum=gbcnet_va_curriculum,
                         do_hierarchical_cls=do_hierarchical_cls, bm_fc_layers=bm_fc_layers).to(device)
params = [p for p in gbcret.parameters() if p.requires_grad]
optimizer = SGD(params, lr=lr, weight_decay=weight_decay, momentum=momentum)
lr_scheduler = None

checkpoint = torch.load("/content/drive/MyDrive/CV/Project/ResNet, GBCNet, GBCRet, 3/GBCRet, layer4 combination, Hierarchical cls, gbcnet VA, 50 epochs pretrained RN, CrossEntropyLoss, dropout=0.5, bm_fc_layers=4/checkpoints/checkpoint_epoch_69.pth", map_location=device)
gbcret.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
last_epoch = checkpoint['epoch']
train_scores = checkpoint['train_scores']
val_scores = checkpoint['val_scores']
best_acc = checkpoint['best_acc']
num_worse_epochs = checkpoint['num_worse_epochs']
random.setstate(checkpoint['random_state'])
np.random.set_state(checkpoint['np_random_state'])
torch.set_rng_state(checkpoint['torch_random_state'].cpu())

train_model(gbcret, optimizer=optimizer, num_epochs=num_epochs, best_acc=best_acc, train_transform=train_transform,
            eval_transform=eval_transform, lr_scheduler=lr_scheduler, va_curriculum=va_curriculum, batch_size=batch_size,
            save_model_dir=save_model_dir, last_epoch=last_epoch, train_scores=train_scores, val_scores=val_scores,
            checkpoint_save_rate=3, num_worse_epochs=num_worse_epochs, patience=30)

Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_v2_coco-5905b1c5.pth
100%|██████████| 146M/146M [00:03<00:00, 43.7MB/s]


Retinanet's COCO pre-trained weights are loaded.
Last Epoch: 69, Start Epoch: 70, End Epoch: 100
VA Curriculum: False
Train Scores Length: 69, Val Scores Length: 69
Best accuracy so far: 0.9098360655737705
Patience so far: 16

Epoch 70:
Number of Train images: 2036
Number of Test images: 122
Gold Curriculum Deactivated.
Training only with gold boxes.
GBCNet VA Curriculum Activated.
Combining ResNet layer4 and GBCNet backbones.
Hierarchical Classification is Activated.
Train Loss: 0.40130255645827245, Total step: 38
Train Evaluation:
mIoU = 0.8184240460395813, median IoU: 0.8332716226577759, mAP = 0.627, mAP_50 = 0.99, mAP_75 = 0.748
TP: 2029, FN: 0, FP: 7
Precision: 0.9965618860510805
Recall: 1.0
Acc: 0.9519 Spec: 0.9954 Sens: 0.9745
Normal vs Not Noraml Acc: 0.9597 Benign vs Malignant Acc: 0.9835
Val Evaluation:
mIoU = 0.6537341475486755, median IoU: 0.7041088342666626, mAP = 0.302, mAP_50 = 0.77, mAP_75 = 0.164
TP: 117, FN: 0, FP: 5
Precision: 0.9590163934426229
Recall: 1.0
Acc: 0.90